##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [1]:
%pip install -q -U "google-genai>=1.0.0"
%pip install -q wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [2]:
from google.colab import userdata
from google import genai
API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

In [3]:
# Define the model to be used in this notebook
# Note: experimental models have more limited quota for API calls.

MODEL_ID="gemini-2.5-flash-preview-05-20" # @param ["gemini-2.5-flash-preview-05-20", "gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}

## Function Calling with the Gemini API

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Step 1: Define the Search Function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-2.0-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [4]:
from typing import List
import textwrap

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

# This is the actual function that would be called based on the model's suggestion
# Define the function with type hints and docstring
def wikipedia_search(search_queries: List[str]) -> List[str]:
  """Search wikipedia for each query and summarize relevant docs.
    Args:
        search_queries: The user query to search wikipedia.

    Returns:
        A list of relevant information from wikipedia based on the query.
  """
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []

  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-2.0-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content

        response = client.models.generate_content(
                model=MODEL_ID,
                contents=textwrap.dedent(f"""
                    Extract relevant information
                    about user's query: {query}
                    From this source:

                    {page}

                    Note: Do not summarize. Only Extract and return the relevant information
                """,
            ),
        )

        urls = [url]

        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue
    print()

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results

In [6]:
example = wikipedia_search(["What is the transformer architecture for LLMs?"])

Searching for "What is the transformer architecture for LLMs?"
Related search terms: ['Attention Is All You Need', 'Large language model', 'Transformer (deep learning architecture)']
Fetching page: "Attention Is All You Need"
Information Source: https://en.wikipedia.org/wiki/Attention_Is_All_You_Need
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Fetching page: "Transformer (deep learning architecture)"
Information Source: https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)

Information Sources:
     https://en.wikipedia.org/wiki/Attention_Is_All_You_Need
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)


Here is what the search results look like:

In [7]:
from IPython.display import display, Markdown

for e in example:
  display(Markdown(e))

The Transformer architecture forms the underlying architecture for most forms of modern Large Language Models (LLMs). A key reason for why the architecture is preferred by most modern LLMs is the parallelizability of the architecture over its predecessors. This ensures that the operations necessary for training can be accelerated on a GPU allowing both faster training times and models of bigger sizes to be trained.

The Transformer architecture introduced the following mechanisms:
*   **Scaled dot-product attention & self-attention**: The use of this mechanism instead of a Recurrent neural network or Long short-term memory allows for better performance. It relies on Query (Q), Key (K), and Value (V) matrices that come from the same source itself (i.e., the input sequence / context window), which eliminates the need for RNNs completely, ensuring parallelizability. The formula for scaled dot-product attention is: Attention(Q,K,V) := softmax((Q × K^T)/sqrt(d_k)) × V, where Q, K, V are the query, key, value matrices, and d_k is the dimension of the values.
*   **Multi-head attention**: This enhances the self-attention mechanism by introducing multiple parallel attention heads. Each head learns different linear projections of the Q, K, and V matrices, allowing the model to capture different aspects of the relationships between words in the sequence simultaneously. This ensures that input embeddings are updated from a more varied and diverse set of perspectives. The outputs from all heads are concatenated and passed through a final linear transformation.
*   **Positional encoding**: Since the Transformer does not rely on the sequence of the text (unlike seq2seq models), it uses sine and cosine wave functions to encode the position of the token into the embedding. These positional encodings are then added to the word embedding at that corresponding position. This method was chosen because it "may allow the model to extrapolate to sequence lengths longer than the ones encountered during training."

Based on:
  https://en.wikipedia.org/wiki/Attention_Is_All_You_Need

*   **Introduction:** At the 2017 NeurIPS conference, Google researchers introduced the transformer architecture in their landmark paper "Attention Is All You Need". This paper's goal was to improve upon 2014 seq2seq technology, and was based mainly on the attention mechanism developed by Bahdanau et al. in 2014.
*   **Components:** Though the original transformer has both encoder and decoder blocks, BERT is an encoder-only model. Academic and research usage of BERT began to decline in 2023, following rapid improvements in the abilities of decoder-only models (such as GPT) to solve tasks via prompting.
*   **Prevalence:** As of 2024, the largest and most capable models are all based on the transformer architecture.
*   **Key Mechanism (Attention):** In order to find out which tokens are relevant to each other within the scope of the context window, the attention mechanism calculates "soft" weights for each token, more precisely for its embedding, by using multiple attention heads, each with its own "relevance" for calculating its own soft weights. For example, the small (i.e. 117M parameter sized) GPT-2 model has had twelve attention heads and a context window of only 1k tokens. In its medium version it has 345M parameters and contains 24 layers, each with 12 attention heads.

Based on:
  https://en.wikipedia.org/wiki/Large_language_model

The transformer is a deep learning architecture, introduced in 2017, that has been widely adopted for training Large Language Models (LLMs). Its core innovation is the multi-head attention mechanism, which allows for parallel processing of text and efficient training compared to earlier recurrent neural networks (RNNs).

For LLMs, the transformer architecture typically includes:

1.  **Tokenizers:** Convert input text into numerical representations called tokens.
2.  **Embedding Layer:** Converts these tokens into vector representations (token embeddings) and adds **positional encodings** to provide information about the tokens' order in the sequence.
3.  **Transformer Layers:** These are stacked and perform repeated transformations on the vector representations to extract linguistic information. Each layer primarily consists of:
    *   **Multi-Head Attention:** A mechanism that allows each token to be contextualized by "mixing" information from other tokens within the context window. It calculates "attention weights" based on learned query, key, and value projections, enabling the model to focus on relevant parts of the input. Multi-head attention allows the model to capture different types of "relevance" simultaneously.
    *   **Feedforward Network (FFN):** A two-layered multilayer perceptron applied independently to each token's representation for further processing.
    *   **Residual Connections and Layer Normalization:** These are crucial for stabilizing training and ensuring numerical stability, allowing for the stacking of many layers.
4.  **Un-embedding Layer:** Converts the final vector representations back into a probability distribution over the vocabulary to generate output tokens.

LLMs often use **adapted variations** of the original encoder-decoder transformer:

*   **Decoder-only Transformers:** Predominantly used for text generation and instruction following (e.g., GPT series, Chinchilla). These models contain a causally masked self-attention mechanism (preventing attention to future tokens) and a feedforward network, but no encoder or cross-attention.
*   **Encoder-only Transformers:** Used for tasks like text embedding and representation learning (e.g., BERT). They process the entire input at once, allowing each token to attend to all other tokens (all-to-all attention).
*   **Encoder-decoder Transformers:** While the original architecture, they are still used for LLMs for tasks like machine translation or complex instruction following (e.g., T5 series). They combine an encoder to process input and a decoder to generate output.

Based on:
  https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)

### Step 2: Pass the System Instructions and Tools to the Model

### Define the prompt

In order to have multiple supporting search queries to the user's original query, you will instruct the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [8]:
instructions = """
    You have access to the Wikipedia API which you will be using
    to answer a user's query. Your job is to generate a list of search queries which
    might answer a user's question. Be creative by using various key-phrases from
    the user's query. To generate variety of queries, ask questions which are
    related to the user's query that might help to find the answer. The more
    queries you generate the better are the odds of you finding the correct answer.
    Here is an example:

    user: Tell me about Cricket World cup 2023 winners.

    function_call: wikipedia_search([
        'What is the name of the team that won the Cricket World Cup 2023?',
        'Who was the captain of the Cricket World Cup 2023 winning team?',
        'Which country hosted the Cricket World Cup 2023?',
        'What was the venue of the Cricket World Cup 2023 final match?',
        'Cricket World cup 2023',
        'Who lifted the Cricket World Cup 2023 trophy?'
    ])

    The search function will return a list of article summaries, use these to
    answer the user's question.
"""

When using the Python SDK, you can provide Python functions directly as tools. The SDK automatically converts the Python function to declarations, handles the function call execution and response cycle for you. The Python SDK then automatically:

- Detects function call responses from the model.
- Call the corresponding Python function in your code.
- Sends the function response back to the model.
- Returns the model's final text response.

To use this, define your function with type hints and a docstring as you did above, and then pass the function itself (not a JSON declaration) as a tool. Note: This approach only handles annotations of `AllowedType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType])`

In order to yield creative and a more random variety of questions, you could also utilize the model's temperature parameter. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

In [9]:
from google.genai import types

config = types.GenerateContentConfig(
    tools=[wikipedia_search],
    system_instruction=instructions,
) # Pass the function itself

### Step 3: Enable Automatic Function Calling and Call the API (Python SDK Only)

Now start a new chat with default value of `automatic_function_calling`. Using the default `disable=None`, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [11]:
chat = client.chats.create(model=MODEL_ID, config=config, history=[])

query = "Compare dog and cat characteristics."

response = chat.send_message(query)

Searching for "Characteristics of dogs"
Related search terms: ['Dog breeding', 'Dog', 'Herding dog']
Fetching page: "Dog breeding"
Information Source: https://en.wikipedia.org/wiki/Dog_breeding
Fetching page: "Dog"
Information Source: https://en.wikipedia.org/wiki/Dog
Fetching page: "Herding dog"
Information Source: https://en.wikipedia.org/wiki/Herding_dog

Searching for "Characteristics of cats"
Related search terms: ['Cat', 'Maine Coon', 'Tabby cat']
Fetching page: "Cat"
Information Source: https://en.wikipedia.org/wiki/Cat
Fetching page: "Maine Coon"
Information Source: https://en.wikipedia.org/wiki/Maine_Coon
Fetching page: "Tabby cat"
Information Source: https://en.wikipedia.org/wiki/Tabby_cat

Searching for "Differences between dogs and cats"
Related search terms: ['Tabby cat', 'Cat people and dog people', 'Cat']
Fetching page: "Cat people and dog people"
Information Source: https://en.wikipedia.org/wiki/Cat_people_and_dog_people

Searching for "Dog behavior vs cat behavior"
Rel

In [12]:
display(Markdown(response.text))

Dogs and cats, while both domesticated mammals, exhibit distinct characteristics in their physical attributes, behavior, and senses.

**Dogs:**

*   **Physical:** Dogs are incredibly diverse in shape, size, and color, with around 450 globally recognized breeds. They range from tiny Chihuahuas to massive Great Danes. Their skeletons are well-adapted for running, and they possess powerful jaws with about 42 teeth. Most dogs have 26 tail vertebrae, and their tails are crucial for communication and balance. Their coats can be double or single, and some breeds are hairless. They have well-developed senses of smell (40 times more receptors than humans), hearing (up to four times greater than humans), and vision (though inferior to humans in acuity, they have dichromatic vision).
*   **Behavioral:** Dogs have been selectively bred for desired behaviors, including guarding, herding, hunting, retrieving, and scent detection. They are highly social animals, uniquely attuned to human behaviors and capable of understanding and communicating with humans. They display complex social cognition and can learn by inference. Their communication includes eye gaze, facial expression, vocalization, and body posture. Dogs typically reach sexual maturity between six months and one year, and females have estrous cycles semi-annually.
*   **Health:** Dogs can be susceptible to various inheritable conditions like hip dysplasia, eye abnormalities, heart conditions, and deafness. Neoplasia (cancer) is a common cause of death. Their median longevity is around 12.7 years, negatively correlated with obesity.

**Cats:**

*   **Physical:** Cats are small carnivorous mammals, typically averaging 46 cm in body length and 23-25 cm in height, with about 30 cm long tails. Males are generally larger than females. They have a smaller skull and shorter bones than their wild ancestors. Their claws are protractible and retractable, keeping them sharp for hunting and climbing. Cats are digitigrade, walking on their toes, and use a "pacing" gait. They have excellent night vision due to a tapetum lucidum and large pupils, but poor color vision. Their acute sense of hearing can detect a broad range of frequencies, and they have a highly developed sense of smell with a Jacobson's organ for sensing pheromones. They have about 470 taste buds and cannot taste sweetness. Whiskers provide information about their surroundings.
*   **Behavioral:** Cats are solitary hunters but social species. They are crepuscular predators, most active at dawn and dusk, though house cats can adapt to human schedules. They conserve energy by sleeping extensively, averaging 12-16 hours a day. Their intelligence is evident in their adaptability, observational learning, and problem-solving skills. Communication includes a variety of vocalizations and body language. They have rigorous grooming habits and instinctively bury waste. Adult cats are thought to live in an extended kittenhood, a form of behavioral neoteny. They mark territories with urine spraying and scent gland rubbing. Cats are known for their love of play, which mimics hunting. Most adult cats are lactose intolerant.
*   **Health:** About 250 heritable genetic disorders have been identified in cats. Neutering can increase life expectancy and confer health benefits but can also lead to obesity. The average lifespan of pet cats has risen to about 13 years as of 2014 and 2023.

**Key Differences and Similarities:**

*   **Domestication:** Dogs have undergone more significant changes in anatomy and behavior during domestication compared to cats, which still retain a strong ability to survive in the wild.
*   **Social Structure:** Dogs are generally more social and attuned to human interaction, while cats, though capable of affection, are inherently more solitary hunters.
*   **Cognition:** Dogs have about twice the number of neurons in their cerebral cortexes than cats, but studies suggest both have similar cognitive abilities to horses and chimpanzees.
*   **Diet:** Dogs are typically described as omnivores and have adapted to starch-rich diets, while cats are obligate carnivores with specific amino acid requirements.
*   **Breeding:** Dog breeding has focused on specific tasks and exaggerated features, leading to greater phenotypic diversity. Cat breeding, which began later, emphasizes aesthetic differences, with fewer genetically distinct breeds.

Check for additional citations:

In [13]:
response.candidates[0].citation_metadata or 'No citations found'

'No citations found'

### Step 4: Understand Chat History

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [14]:
import json
for content in chat.get_history():
    display(Markdown("### " + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print(part.function_call.name, json.dumps(part.function_call.args, indent=2))
        if part.function_response:
            for res in part.function_response.response['result']:
                display(Markdown(res))
    print("-" * 80)

### user:

Compare dog and cat characteristics.

--------------------------------------------------------------------------------


### model:

wikipedia_search {
  "search_queries": [
    "Characteristics of dogs",
    "Characteristics of cats",
    "Differences between dogs and cats",
    "Dog behavior vs cat behavior",
    "Dog breeds and their traits",
    "Cat breeds and their traits",
    "Common traits of canines",
    "Common traits of felines"
  ]
}
--------------------------------------------------------------------------------


...

The provided text does not contain information about "Common traits of felines." It is an introduction to a list of fictional feline characters from animated shows and films.

Based on:
  https://en.wikipedia.org/wiki/List_of_fictional_cats_in_animation

--------------------------------------------------------------------------------


### model:

Dogs and cats, while both domesticated mammals, exhibit distinct characteristics in their physical attributes, behavior, and senses.

**Dogs:**

*   **Physical:** Dogs are incredibly diverse in shape, size, and color, with around 450 globally recognized breeds. They range from tiny Chihuahuas to massive Great Danes. Their skeletons are well-adapted for running, and they possess powerful jaws with about 42 teeth. Most dogs have 26 tail vertebrae, and their tails are crucial for communication and balance. Their coats can be double or single, and some breeds are hairless. They have well-developed senses of smell (40 times more receptors than humans), hearing (up to four times greater than humans), and vision (though inferior to humans in acuity, they have dichromatic vision).
*   **Behavioral:** Dogs have been selectively bred for desired behaviors, including guarding, herding, hunting, retrieving, and scent detection. They are highly social animals, uniquely attuned to human behaviors and capable of understanding and communicating with humans. They display complex social cognition and can learn by inference. Their communication includes eye gaze, facial expression, vocalization, and body posture. Dogs typically reach sexual maturity between six months and one year, and females have estrous cycles semi-annually.
*   **Health:** Dogs can be susceptible to various inheritable conditions like hip dysplasia, eye abnormalities, heart conditions, and deafness. Neoplasia (cancer) is a common cause of death. Their median longevity is around 12.7 years, negatively correlated with obesity.

**Cats:**

*   **Physical:** Cats are small carnivorous mammals, typically averaging 46 cm in body length and 23-25 cm in height, with about 30 cm long tails. Males are generally larger than females. They have a smaller skull and shorter bones than their wild ancestors. Their claws are protractible and retractable, keeping them sharp for hunting and climbing. Cats are digitigrade, walking on their toes, and use a "pacing" gait. They have excellent night vision due to a tapetum lucidum and large pupils, but poor color vision. Their acute sense of hearing can detect a broad range of frequencies, and they have a highly developed sense of smell with a Jacobson's organ for sensing pheromones. They have about 470 taste buds and cannot taste sweetness. Whiskers provide information about their surroundings.
*   **Behavioral:** Cats are solitary hunters but social species. They are crepuscular predators, most active at dawn and dusk, though house cats can adapt to human schedules. They conserve energy by sleeping extensively, averaging 12-16 hours a day. Their intelligence is evident in their adaptability, observational learning, and problem-solving skills. Communication includes a variety of vocalizations and body language. They have rigorous grooming habits and instinctively bury waste. Adult cats are thought to live in an extended kittenhood, a form of behavioral neoteny. They mark territories with urine spraying and scent gland rubbing. Cats are known for their love of play, which mimics hunting. Most adult cats are lactose intolerant.
*   **Health:** About 250 heritable genetic disorders have been identified in cats. Neutering can increase life expectancy and confer health benefits but can also lead to obesity. The average lifespan of pet cats has risen to about 13 years as of 2014 and 2023.

**Key Differences and Similarities:**

*   **Domestication:** Dogs have undergone more significant changes in anatomy and behavior during domestication compared to cats, which still retain a strong ability to survive in the wild.
*   **Social Structure:** Dogs are generally more social and attuned to human interaction, while cats, though capable of affection, are inherently more solitary hunters.
*   **Cognition:** Dogs have about twice the number of neurons in their cerebral cortexes than cats, but studies suggest both have similar cognitive abilities to horses and chimpanzees.
*   **Diet:** Dogs are typically described as omnivores and have adapted to starch-rich diets, while cats are obligate carnivores with specific amino acid requirements.
*   **Breeding:** Dog breeding has focused on specific tasks and exaggerated features, leading to greater phenotypic diversity. Cat breeding, which began later, emphasizes aesthetic differences, with fewer genetically distinct breeds.

--------------------------------------------------------------------------------


In the chat history you can see all 4 steps:

1. User: Asks questions about cats and dogs.
2. Model: Determines that the `wikipedia_search` is helpful and sends a FunctionCall request to the user.
3. User: The Chat session automatically executes the function (due to `_automatic_function_calling` is enabled by default) and sends back a FunctionResponse with the searched result.
4. Model: Uses the function's output to formulate the final answer and presents it as a text response.

## [Optional] Manually Execute the Function Call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [15]:
config  = {
        "tools": [wikipedia_search],
        "automatic_function_calling": {"disable": True}, # for manual execution
    }

chat    = client.chats.create(model=MODEL_ID, config=config, history=[])
response  = chat.send_message(query)

Initially the model returns a `FunctionCall`:

In [16]:
fc = response.candidates[0].content.parts[0].function_call
print(json.dumps(fc.args, indent=2))

{
  "search_queries": [
    "dog",
    "cat"
  ]
}


Call the function with generated arguments to get the results.

In [17]:
if fc.name == "wikipedia_search":
    summaries = wikipedia_search(**fc.args)
    display(Markdown("\n ### Function execution result:"))
    for text in summaries:
        display(Markdown(text))

Searching for "dog"
Related search terms: ['Dog', 'Dog (disambiguation)', 'Dog Eat Dog']
Fetching page: "Dog"
Information Source: https://en.wikipedia.org/wiki/Dog
Fetching page: "Dog (disambiguation)"
Results when searching for "Dog (disambiguation)" (originally for "dog")
        were ambiguous, hence skipping
Fetching page: "Dog Eat Dog"
Results when searching for "Dog Eat Dog" (originally for "dog")
        were ambiguous, hence skipping

Searching for "cat"
Related search terms: ['Cat', 'Cat (disambiguation)', "Schrödinger's cat"]
Fetching page: "Cat"
Information Source: https://en.wikipedia.org/wiki/Cat
Fetching page: "Cat (disambiguation)"
Results when searching for "Cat (disambiguation)" (originally for "cat")
        were ambiguous, hence skipping
Fetching page: "Schrödinger's cat"
Information Source: https://en.wikipedia.org/wiki/Schr%C3%B6dinger%27s_cat

Information Sources:
     https://en.wikipedia.org/wiki/Dog
     https://en.wikipedia.org/wiki/Cat
     https://en.wikiped


 ### Function execution result:

The dog (Canis familiaris or Canis lupus familiaris) is a domesticated descendant of the gray wolf.
Also called the domestic dog, it was selectively bred from an extinct population of wolves during the Late Pleistocene by hunter-gatherers.
The dog was the first species to be domesticated by humans, over 14,000 years ago and before the development of agriculture.
Due to their long association with humans, dogs have gained the ability to thrive on a starch-rich diet that would be inadequate for other canids.
Dogs have been bred for desired behaviors, sensory capabilities, and physical attributes.
Dog breeds vary widely in shape, size, and color.
They have the same number of bones (with the exception of the tail), powerful jaws that house around 42 teeth, and well-developed senses of smell, hearing, and sight.
Compared to humans, dogs possess a superior sense of smell and hearing, but inferior visual acuity.
Dogs perform many roles for humans, such as hunting, herding, pulling loads, protection, companionship, therapy, aiding disabled people, and assisting police and the military.
Communication in dogs includes eye gaze, facial expression, vocalization, body posture (including movements of bodies and limbs), and gustatory communication (scents, pheromones, and taste).
They mark their territories by urinating on them, which is more likely when entering a new environment.
Over the millennia, dogs have uniquely adapted to human behavior; this adaptation includes being able to understand and communicate with humans.
As such, the human–canine bond has been a topic of frequent study, and dogs' influence on human society has given them the sobriquet of "man's best friend".
The global dog population is estimated at 700 million to 1 billion, distributed around the world.
The dog is the most popular pet in the United States, present in 34–40% of households.
Developed countries make up approximately 20% of the global dog population, while around 75% of dogs are estimated to be from developing countries, mainly in the form of feral and community dogs.
Dogs are domesticated members of the family Canidae.
They are classified as a subspecies of Canis lupus, along with wolves and dingoes.
Dogs were domesticated from wolves over 14,000 years ago by hunter-gatherers, before the development of agriculture.
The remains of the Bonn–Oberkassel dog, buried alongside humans between 14,000 and 15,000 years ago, are the earliest to be conclusively identified as a domesticated dog.
Genetic studies show that dogs likely diverged from wolves between 27,000 and 40,000 years ago.
The dingo and the related New Guinea singing dog resulted from the geographic isolation and feralization of dogs in Oceania over 8,000 years ago.
Dogs, wolves, and dingoes have sometimes been classified as separate species.
In 1758, the Swedish botanist and zoologist Carl Linnaeus assigned the genus name Canis (which is the Latin word for "dog") to the domestic dog, the wolf, and the golden jackal in his book, Systema Naturae.
He classified the domestic dog as Canis familiaris.
Linnaeus considered the dog to be a separate species from the wolf because of its upturning tail (cauda recurvata in Latin term), which is not found in any other canid.
In the 2005 edition of Mammal Species of the World, mammalogist W. Christopher Wozencraft listed the wolf as a wild subspecies of Canis lupus and proposed two additional subspecies: familiaris, as named by Linnaeus in 1758, and dingo, named by Meyer in 1793.
Wozencraft included hallstromi (the New Guinea singing dog) as another name (junior synonym) for the dingo.
The earliest remains generally accepted to be those of a domesticated dog were discovered in Bonn-Oberkassel, Germany.
Contextual, isotopic, genetic, and morphological evidence shows that this dog was not a local wolf.
The dog was dated to 14,223 years ago and was found buried along with a man and a woman, all three having been sprayed with red hematite powder and buried under large, thick basalt blocks.
The dog had died of canine distemper.
This timing indicates that the dog was the first species to be domesticated in the time of hunter-gatherers, which predates agriculture.
DNA sequences show that all ancient and modern dogs share a common ancestry and descended from an ancient, extinct wolf population that was distinct from any modern wolf lineage.
The dog is a domestic animal that likely travelled a commensal pathway into domestication (i.e. humans initially neither benefitted nor were harmed by wild dogs eating refuse from their camps).
In 2021, a literature review of the current evidence infers that the dog was domesticated in Siberia 23,000 years ago by ancient North Siberians, then later dispersed eastward into the Americas and westward across Eurasia, with dogs likely accompanying the first humans to inhabit the Americas.
In 2018, a study identified 429 genes that differed between modern dogs and modern wolves.
As the differences in these genes could also be found in ancient dog fossils, these were regarded as being the result of the initial domestication and not from recent breed formation.
These genes are linked to neural crest and central nervous system development.
These genes affect embryogenesis and can confer tameness, smaller jaws, floppy ears, and diminished craniofacial development, which distinguish domesticated dogs from wolves and are considered to reflect domestication syndrome.
The study concluded that during early dog domestication, the initial selection was for behavior.
This trait is influenced by those genes which act in the neural crest, which led to the phenotypes observed in modern dogs.
There are around 450 official dog breeds, the most of any mammal.
Dogs began diversifying in the Victorian era, when humans took control of their natural selection.
Most breeds were derived from small numbers of founders within the last 200 years.
Since then, dogs have undergone rapid phenotypic change and have been subjected to artificial selection by humans.
The skull, body, and limb proportions between breeds display more phenotypic diversity than can be found within the entire order of carnivores.
These breeds possess distinct traits related to morphology, which include body size, skull shape, tail phenotype, fur type, and colour.
As such, humans have long used dogs for their desirable traits to complete or fulfill a certain work or role.
Their behavioural traits include guarding, herding, hunting, retrieving, and scent detection.
Their personality traits include hypersocial behavior, boldness, and aggression.
Present-day dogs are dispersed around the world.
Dogs are extremely variable in size, ranging from one of the largest breeds, the Great Dane, at 50 to 79 kg (110 to 174 lb) and 71 to 81 cm (28 to 32 in), to one of the smallest, the Chihuahua, at 0.5 to 3 kg (1.1 to 6.6 lb) and 13 to 20 cm (5.1 to 7.9 in).
All healthy dogs, regardless of their size and type, have the same amount of bones (with the exception of the tail), although there is significant skeletal variation between dogs of different types.
The dog's skeleton is well adapted for running; the vertebrae on the neck and back have extensions for back muscles, consisting of epaxial muscles and hypaxial muscles, to connect to; the long ribs provide room for the heart and lungs; and the shoulders are unattached to the skeleton, allowing for flexibility.
Compared to the dog's wolf-like ancestors, selective breeding since domestication has seen the dog's skeleton increase in size for larger types such as mastiffs and miniaturised for smaller types such as terriers.
Most dogs naturally have 26 vertebrae in their tails, but some with naturally short tails have as few as three.
The dog's skull has identical components regardless of breed type, but there is significant divergence in terms of skull shape between types.
The three basic skull shapes are the elongated dolichocephalic type as seen in sighthounds, the intermediate mesocephalic or mesaticephalic type, and the very short and broad brachycephalic type exemplified by mastiff type skulls.
The jaw contains around 42 teeth, and it has evolved for the consumption of flesh.
Dogs use their carnassial teeth to cut food into bite-sized chunks, more especially meat.
Dogs' senses include vision, hearing, smell, taste, touch, and magnetoreception.
One study suggests that dogs can feel small variations in Earth's magnetic field.
Dogs prefer to defecate with their spines aligned in a north–south position in calm magnetic field conditions.
Dogs' vision is dichromatic; their visual world consists of yellows, blues, and grays.
They have difficulty differentiating between red and green, and much like other mammals, the dog's eye is composed of two types of cone cells compared to the human's three.
The divergence of the eye axis of dogs ranges from 12 to 25°, depending on the breed, which can have different retina configurations.
The fovea centralis area of the eye is attached to a nerve fiber, and is the most sensitive to photons.
Additionally, a study found that dogs' visual acuity was up to eight times less effective than a human, and their ability to discriminate levels of brightness was about two times worse than a human.
While the human brain is dominated by a large visual cortex, the dog brain is dominated by a large olfactory cortex.
Dogs have roughly forty times more smell-sensitive receptors than humans, ranging from about 125 million to nearly 300 million in some dog breeds, such as bloodhounds.
This sense of smell is the most prominent sense of the species; it detects chemical changes in the environment, allowing dogs to pinpoint the location of mating partners, potential stressors, resources, etc.
Dogs also have an acute sense of hearing up to four times greater than that of humans.
They can pick up the slightest sounds from about 400 m (1,300 ft) compared to 90 m (300 ft) for humans.
Dogs have stiff, deeply embedded hairs known as whiskers that sense atmospheric changes, vibrations, and objects not visible in low light conditions.
The lower most part of whiskers hold more receptor cells than other hair types, which help in alerting dogs of objects that could collide with the nose, ears, and jaw.
Whiskers likely also facilitate the movement of food towards the mouth.
The coats of domestic dogs are of two varieties: "double" being common in dogs (as well as wolves) originating from colder climates, made up of a coarse guard hair and a soft down hair, or "single", with the topcoat only.
Breeds may have an occasional "blaze", stripe, or "star" of white fur on their chest or underside.
Premature graying can occur in dogs as early as one year of age; this is associated with impulsive behaviors, anxiety behaviors, and fear of unfamiliar noise, people, or animals.
Some dog breeds are hairless, while others have a very thick corded coat.
A dog's dewclaw is the fifth digit in its forelimb and hind legs.
Dewclaws on the forelimbs are attached by bone and ligament, while the dewclaws on the hind legs are attached only by skin.
Most dogs are not born with dewclaws in their hind legs, and some are without them in their forelimbs.
Dogs' dewclaws consist of the proximal phalanges and distal phalanges.
A dog's tail is the terminal appendage of the vertebral column, which is made up of a string of 5 to 23 vertebrae enclosed in muscles and skin that support the dog's back extensor muscles.
One of the primary functions of a dog's tail is to communicate their emotional state.
The tail also helps the dog maintain balance by putting its weight on the opposite side of the dog's tilt, and it can also help the dog spread its anal gland's scent through the tail's position and movement.
Dogs can have a violet gland (or supracaudal gland) characterized by sebaceous glands on the dorsal surface of their tails; in some breeds, it may be vestigial or absent.
A study suggests that dogs show asymmetric tail-wagging responses to different emotive stimuli.
Dogs can injure themselves by wagging their tails forcefully; this condition is called kennel tail, happy tail, bleeding tail, or splitting tail.
In some hunting dogs, the tail is traditionally docked to avoid injuries.
Some dogs can be born without tails because of a DNA variant in the T gene, which can also result in a congenitally short (bobtail) tail.
Numerous disorders have been known to affect dogs.
Some are congenital and others are acquired.
Dogs can acquire upper respiratory tract diseases including diseases that affect the nasal cavity, the larynx, and the trachea; lower respiratory tract diseases which includes pulmonary disease and acute respiratory diseases; heart diseases which includes any cardiovascular inflammation or dysfunction of the heart; haemopoietic diseases including anaemia and clotting disorders; gastrointestinal disease such as diarrhoea and gastric dilatation volvulus; hepatic disease such as portosystemic shunts and liver failure; pancreatic disease such as pancreatitis; renal disease; lower urinary tract disease such as cystitis and urolithiasis; endocrine disorders such as diabetes mellitus, Cushing's syndrome, hypoadrenocorticism, and hypothyroidism; nervous system diseases such as seizures and spinal injury; musculoskeletal disease such as arthritis and myopathies; dermatological disorders such as alopecia and pyoderma; ophthalmological diseases such as conjunctivitis, glaucoma, entropion, and progressive retinal atrophy; and neoplasia.
Common dog parasites are lice, fleas, fly larvae, ticks, mites, cestodes, nematodes, and coccidia.
Taenia is a notable genus with 5 species in which dogs are the definitive host.
Additionally, dogs are a source of zoonoses for humans.
They are responsible for 99% of rabies cases worldwide.
Other common zoonoses are hydatid disease, leptospirosis, pasteurellosis, ringworm, and toxocariasis.
Common infections in dogs include canine adenovirus, canine distemper virus, canine parvovirus, leptospirosis, canine influenza, and canine coronavirus.
All of these conditions have vaccines available.
Dogs are the companion animal most frequently reported for exposure to toxins.
Most poisonings are accidental and over 80% of reports of exposure to the ASPCA animal poisoning hotline are due to oral exposure.
Neoplasia is the most common cause of death for dogs.
Other common causes of death are heart and renal failure.
Their pathology is similar to that of humans, as is their response to treatment and their outcomes.
Genes found in humans to be responsible for disorders are investigated in dogs as being the cause and vice versa.
The typical lifespan of dogs varies widely among breeds, but the median longevity (the age at which half the dogs in a population have died and half are still alive) is approximately 12.7 years.
Obesity correlates negatively with longevity with one study finding obese dogs to have a life expectancy approximately a year and a half less than dogs with a healthy weight.
In a 2024 UK study analyzing 584,734 dogs, it was concluded that purebred dogs lived longer than crossbred dogs, challenging the previous notion of the latter having the higher life expectancies.
Other studies also show that fully mongrel dogs live about a year longer on average than dogs with pedigrees.
Furthermore, small dogs with longer muzzles have been shown to have higher lifespans than larger medium-sized dogs with much more depressed muzzles.
For free-ranging dogs, less than 1 in 5 reach sexual maturity, and the median life expectancy for feral dogs is less than half of dogs living with humans.
In domestic dogs, sexual maturity happens around six months to one year for both males and females, although this can be delayed until up to two years of age for some large breeds.
This is the time at which female dogs will have their first estrous cycle, characterized by their vulvas swelling and producing discharges, usually lasting between 4 and 20 days.
They will experience subsequent estrous cycles semiannually, during which the body prepares for pregnancy.
At the peak of the cycle, females will become estrous, mentally and physically receptive to copulation.
Because the ova survive and can be fertilized for a week after ovulation, more than one male can sire the same litter.
Fertilization typically occurs two to five days after ovulation.
After ejaculation, the dogs are coitally tied for around 5–30 minutes because of the male's bulbus glandis swelling and the female's constrictor vestibuli contracting; the male will continue ejaculating until they untie naturally due to muscle relaxation.
14–16 days after ovulation, the embryo attaches to the uterus, and after seven to eight more days, a heartbeat is detectable.
Dogs bear their litters roughly 58 to 68 days after fertilization, with an average of 63 days, although the length of gestation can vary.
An average litter consists of about six puppies.
Neutering is the sterilization of animals via gonadectomy, which is an orchidectomy (castration) in dogs and ovariohysterectomy (spay) in bitches.
Neutering reduces problems caused by hypersexuality, especially in male dogs.
Spayed females are less likely to develop cancers affecting the mammary glands, ovaries, and other reproductive organs.
However, neutering increases the risk of urinary incontinence in bitches, prostate cancer in dogs, and osteosarcoma, hemangiosarcoma, cruciate ligament rupture, pyometra, obesity, and diabetes mellitus in either sex.
Neutering is the most common surgical procedure in dogs less than a year old in the US and is seen as a control method for overpopulation.
Neutering often occurs as early as 6–14 weeks in shelters in the US.
The American Society for the Prevention of Cruelty to Animals (ASPCA) advises that dogs not intended for further breeding should be neutered so that they do not have undesired puppies that may later be euthanized.
Neutering is less common in most European countries, especially in Nordic countries—except for the UK, where it is common.
In a study of seven dog breeds (the Bernese Mountain Dog, Basset Hound, Cairn Terrier, Brittany, German Shepherd Dog, Leonberger, and West Highland White Terrier), it was found that inbreeding decreases litter size and survival.
Another analysis of data on 42,855 Dachshund litters found that as the inbreeding coefficient increased, litter size decreased and the percentage of stillborn puppies increased, thus indicating inbreeding depression.
In a study of Boxer litters, 22% of puppies died before reaching 7 weeks of age.
Stillbirth was the most frequent cause of death, followed by infection.
Mortality due to infection increased significantly with increases in inbreeding.
Dog behavior has been shaped by millennia of contact with humans.
They have acquired the ability to understand and communicate with humans and are uniquely attuned to human behaviors.
Behavioral scientists suggest that a set of social-cognitive abilities in domestic dogs that are not possessed by the dog's canine relatives or other highly intelligent mammals, such as great apes, are parallel to children's social-cognitive skills.
Dogs have about twice the number of neurons in their cerebral cortexes than what cats have, which suggests they could be about twice as intelligent.
Most domestic animals were initially bred for the production of goods.
Dogs, on the other hand, were selectively bred for desirable behavioral traits.
In 2016, a study found that only 11 fixed genes showed variation between wolves and dogs.
These gene variations indicate the occurrence of artificial selection and the subsequent divergence of behavior and anatomical features.
These genes have been shown to affect the catecholamine synthesis pathway, with the majority of the genes affecting the fight-or-flight response (i.e., selection for tameness) and emotional processing.
Compared to their wolf counterparts, dogs tend to be less timid and less aggressive, though some of these genes have been associated with aggression in certain dog breeds.
Traits of high sociability and lack of fear in dogs may include genetic modifications related to Williams-Beuren syndrome in humans, which cause hypersociability at the expense of problem-solving ability.
In a 2023 study of 58 dogs, some dogs classified as attention deficit hyperactivity disorder-like showed lower serotonin and dopamine concentrations.
A similar study claims that hyperactivity is more common in male and young dogs.
A dog can become aggressive because of trauma or abuse, fear or anxiety, territorial protection, or protecting an item it considers valuable.
Acute stress reactions from post-traumatic stress disorder (PTSD) seen in dogs can evolve into chronic stress.
Police dogs with PTSD can often refuse to work.
Dogs have a natural instinct called prey drive (the term is chiefly used to describe training dogs' habits) which can be influenced by breeding.
These instincts can drive dogs to consider objects or other animals to be prey or drive possessive behavior.
These traits have been enhanced in some breeds so that they may be used to hunt and kill vermin or other pests.
Puppies or dogs sometimes bury food underground.
Some puppies and dogs engage in coprophagy out of habit, stress, for attention, or boredom; most of them will not do it later in life.
A study hypothesizes that the behavior was inherited from wolves, a behavior likely evolved to lessen the presence of intestinal parasites in dens.
Most dogs can swim.
In a study of 412 dogs, around 36.5% of the dogs could not swim; the other 63.5% were able to swim without a trainer in a swimming pool.
A study of 55 dogs found a correlation between swimming and 'improvement' of the hip osteoarthritis joint.
The female dog may produce colostrum, a type of milk high in nutrients and antibodies, 1–7 days before giving birth.
Milk production lasts for around three months, and increases with litter size.
The dog can sometimes vomit and refuse food during child contractions.
In the later stages of the dog's pregnancy, nesting behaviour may occur.
Puppies are born with a protective fetal membrane that the mother usually removes shortly after birth.
Dogs can have the maternal instincts to start grooming their puppies, consume their puppies' feces, and protect their puppies, likely due to their hormonal state.
While male-parent dogs can show more disinterested behaviour toward their own puppies, most can play with the young pups as they would with other dogs or humans.
A female dog may abandon or attack her puppies or her male partner dog if she is stressed or in pain.
Researchers have tested dogs' ability to perceive information, retain it as knowledge, and apply it to solve problems.
Studies of two dogs suggest that dogs can learn by inference.
A study with Rico, a Border Collie, showed that he knew the labels of over 200 different items.
He inferred the names of novel things by exclusion learning and correctly retrieved those new items after four weeks of the initial exposure.
A study of another Border Collie, Chaser, documented that he had learned the names and could associate them by verbal command with over 1,000 words.
One study of canine cognitive abilities found that dogs' capabilities are similar to those of horses, chimpanzees, or cats.
One study of 18 household dogs found that the dogs could not distinguish food bowls at specific locations without distinguishing cues; the study stated that this indicates a lack of spatial memory.
A study stated that dogs have a visual sense for number.
The dogs showed a ratio-dependent activation both for numerical values from 1–3 to larger than four.
Dogs demonstrate a theory of mind by engaging in deception.
Another experimental study showed evidence that Australian dingos can outperform domestic dogs in non-social problem-solving, indicating that domestic dogs may have lost much of their original problem-solving abilities once they joined humans.
Another study showed that dogs stared at humans after failing to complete an impossible version of the same task they had been trained to solve.
Dog communication is the transfer of information between dogs, as well as between dogs and humans.
Communication behaviors of dogs include eye gaze, facial expression, vocalization, body posture (including movements of bodies and limbs), and gustatory communication (scents, pheromones, and taste).
Dogs mark their territories by urinating on them, which is more likely when entering a new environment.
Both sexes of dogs may also urinate to communicate anxiety or frustration, submissiveness, or when in exciting or relaxing situations.
Aroused dogs can be a result of the dogs' higher cortisol levels.
Dogs begin socializing with other dogs by the time they reach the ages of 3 to 8 weeks, and at about 5 to 12 weeks of age, they alter their focus from dogs to humans.
Belly exposure in dogs can be a defensive behavior that can lead to a bite or to seek comfort.
Humans communicate with dogs by using vocalization, hand signals, and body posture.
With their acute sense of hearing, dogs rely on the auditory aspect of communication for understanding and responding to various cues, including the distinctive barking patterns that convey different messages.
A study using functional magnetic resonance imaging (fMRI) has shown that dogs respond to both vocal and nonvocal voices using the brain's region towards the temporal pole, similar to that of humans' brains.
Most dogs also looked significantly longer at the face whose expression matched the valence of vocalization.
A study of caudate responses shows that dogs tend to respond more positively to social rewards than to food rewards.
The dog is the most widely abundant large carnivoran living in the human environment.
In 2020, the estimated global dog population was between 700 million and 1 billion.
In the same year, a study found the dog to be the most popular pet in the United States, as they were present in 34 out of every 100 homes.
About 20% of the dog population live in developed countries.
In the developing world, it is estimated that three-quarters of the world's dog population lives in the developing world as feral, village, or community dogs.
Most of these dogs live as scavengers and have never been owned by humans, with one study showing that village dogs' most common response when approached by strangers is to run away (52%) or respond aggressively (11%).
Feral and free-ranging dogs' potential to compete with other large carnivores is limited by their strong association with humans.
Although wolves are known to kill dogs, wolves tend to live in pairs in areas where they are highly persecuted, giving them a disadvantage when facing large dog groups.
Coyotes and big cats have also been known to attack dogs.
In particular, leopards are known to have a preference for dogs and have been recorded to kill and consume them, no matter their size.
Siberian tigers in the Amur river region have killed dogs in the middle of villages.
Striped hyenas are known to kill dogs in their range.
Dogs as introduced predators have affected the ecology of New Zealand, which lacked indigenous land-based mammals before human settlement.
Dogs have made 11 vertebrate species extinct and are identified as a 'potential threat' to at least 188 threatened species worldwide.
Dogs have also been linked to the extinction of 156 animal species.
Dogs have been documented to have killed a few birds of the endangered species, the kagu, in New Caledonia.
Dogs are typically described as omnivores.
Compared to wolves, dogs from agricultural societies have extra copies of amylase and other genes involved in starch digestion that contribute to an increased ability to thrive on a starch-rich diet.
Similar to humans, some dog breeds produce amylase in their saliva and are classified as having a high-starch diet.
Despite being an omnivore, dogs are only able to conjugate bile acid with taurine.
They must get vitamin D from their diet.
Of the twenty-one amino acids common to all life forms (including selenocysteine), dogs cannot synthesize ten: arginine, histidine, isoleucine, leucine, lysine, methionine, phenylalanine, threonine, tryptophan, and valine.
Like cats, dogs require arginine to maintain nitrogen balance.
These nutritional requirements place dogs halfway between carnivores and omnivores.
As a domesticated or semi-domesticated animal, the dog has notable exceptions of presence in: The Aboriginal Tasmanians, who were separated from Australia before the arrival of dingos on that continent; The Andamanese peoples, who were isolated when rising sea levels covered the land bridge to Myanmar; The Fuegians, who instead domesticated the Fuegian dog, an already extinct different canid species; Individual Pacific islands whose maritime settlers did not bring dogs or where the dogs died out after original settlement, notably the Mariana Islands, Palau and most of the Caroline Islands with exceptions such as Fais Island and Nukuoro, the Marshall Islands, the Gilbert Islands, New Caledonia, Vanuatu, Tonga, Marquesas, Mangaia in the Cook Islands, Rapa Iti in French Polynesia, Easter Island, the Chatham Islands, and Pitcairn Island.
Dogs were introduced to Antarctica as sled dogs.
Starting practice in December 1993, dogs were later outlawed by the Protocol on Environmental Protection to the Antarctic Treaty international agreement due to the possible risk of spreading infections.
The domesticated dog originated as a predator and scavenger.
They inherited complex behaviors, such as bite inhibition, from their wolf ancestors, which would have been pack hunters with complex body language.
These sophisticated forms of social cognition and communication may account for dogs' trainability, playfulness, and ability to fit into human households and social situations, and probably also their co-existence with early human hunter-gatherers.
Dogs perform many roles for people, such as hunting, herding, pulling loads, protection, assisting police and the military, companionship, and aiding disabled individuals.
These roles in human society have earned them the nickname "man's best friend" in the Western world.
In some cultures, however, dogs are also a source of meat.
The keeping of dogs as companions, particularly by elites, has a long history.
Pet-dog populations grew significantly after World War II as suburbanization increased.
In the 1980s, there have been changes in the pet dog's functions, such as the increased role of dogs in the emotional support of their human guardians.
Within the second half of the 20th century, more and more dog owners considered their animal to be a part of the family.
This major social status shift allowed the dog to conform to social expectations of personality and behavior.
The second has been the broadening of the concepts of family and the home to include dogs-as-dogs within everyday routines and practices.
Products such as dog-training books, classes, and television programs target dog owners.
Some dog-trainers have promoted a dominance model of dog-human relationships.
Human family members have increased participation in activities in which the dog is an integral partner, such as dog dancing and dog yoga.
According to statistics published by the American Pet Products Manufacturers Association in the National Pet Owner Survey in 2009–2010, an estimated 77.5 million people in the United States have pet dogs.
The source shows that nearly 40% of American households own at least one dog, of which 67% own just one dog, 25% own two dogs, and nearly 9% own more than two dogs.
The data also shows an equal number of male and female pet dogs; less than one-fifth of the owned dogs come from shelters.
In addition to dogs' role as companion animals, dogs have been bred for herding livestock (such as collies and sheepdogs); for hunting; for rodent control (such as terriers); as search and rescue dogs; as detection dogs (such as those trained to detect illicit drugs or chemical weapons); as homeguard dogs; as police dogs (sometimes nicknamed "K-9"); as welfare-purpose dogs; as dogs who assist fishermen retrieve their nets; and as dogs that pull loads (such as sled dogs).
In 1957, the dog Laika became one of the first animals to be launched into Earth orbit aboard the Soviets's Sputnik 2; Laika died during the flight from overheating.
Various kinds of service dogs and assistance dogs, including guide dogs, hearing dogs, mobility assistance dogs, and psychiatric service dogs, assist individuals with disabilities.
A study of 29 dogs found that 9 dogs owned by people with epilepsy were reported to exhibit attention-getting behavior to their handler 30 seconds to 45 minutes prior to an impending seizure; there was no significant correlation between the patients' demographics, health, or attitude towards their pets.
Dogs compete in breed-conformation shows and dog sports (including racing, sledding, and agility competitions).
In dog shows, also referred to as "breed shows", a judge familiar with the specific dog breed evaluates individual purebred dogs for conformity with their established breed type as described in a breed standard.
Weight pulling, a dog sport involving pulling weight, has been criticized for promoting doping and for its risk of injury.
Humans have consumed dog meat going back at least 14,000 years.
For centuries, the practice was prevalent in Southeast Asia, East Asia, Africa, and Oceania before cultural changes triggered by the spread of religions resulted in dog meat consumption declining and becoming more taboo.
Switzerland, Polynesia, and pre-Columbian Mexico historically consumed dog meat.
Some Native American dogs, like the Peruvian Hairless Dog and Xoloitzcuintle, were raised to be sacrificed and eaten.
Han Chinese traditionally ate dogs.
Consumption of dog meat declined but did not end during the Sui dynasty (581–618) and Tang dynasty (618–907) due in part to the spread of Buddhism and the upper class rejecting the practice.
Dog consumption was rare in India, Iran, and Europe.
Eating dog meat is a social taboo in most parts of the world, though some still consume it in modern times.
It is still consumed in some East Asian countries, including China, Vietnam, Korea, Indonesia, and the Philippines.
An estimated 30 million dogs are killed and consumed in Asia every year.
China is the world's largest consumer of dogs, with an estimated 10 to 20 million dogs killed every year for human consumption.
In Vietnam, about 5 million dogs are slaughtered annually.
In 2024, China, Singapore, and Thailand placed a ban on the consumption of dogs within their borders.
In some parts of Poland and Central Asia, dog fat is reportedly believed to be beneficial for the lungs.
There is a long history of dog meat consumption in South Korea, but the practice has fallen out of favor.
On 9 January 2024, the South Korean parliament passed a law banning the distribution and sale of dog meat. It will take effect in 2027, with plans to assist dog farmers in transitioning to other products.
The primary type of dog raised for meat in South Korea has been the Nureongi.
In North Korea where meat is scarce, eating dog is a common and accepted practice, officially promoted by the government.
In 2018, the World Health Organization (WHO) reported that 59,000 people died globally from rabies, with 59.6% of the deaths in Asia and 36.4% in Africa. Rabies is a disease for which dogs are the most significant vector.
Dog bites affect tens of millions of people globally each year.
The primary victims of dog bite incidents are children.
They are more likely to sustain more serious injuries from bites, which can lead to death.
Sharp claws can lacerate flesh and cause serious infections.
In the United States, cats and dogs are a factor in more than 86,000 falls each year.
It has been estimated that around 2% of dog-related injuries treated in U.K. hospitals are domestic accidents.
The same study concluded that dog-associated road accidents involving injuries more commonly involve two-wheeled vehicles.
Some countries and cities have also banned or restricted certain dog breeds, usually for safety concerns.
Toxocara canis (dog roundworm) eggs in dog feces can cause toxocariasis.
Untreated toxocariasis can cause retinal damage and decreased vision.
Dog feces can also contain hookworms that cause cutaneous larva migrans in humans.
The scientific evidence is mixed as to whether a dog's companionship can enhance human physical and psychological well-being.
Studies suggest that there are benefits to physical health and psychological well-being, but they have been criticized for being "poorly controlled".
Earlier studies have shown that pet-dog or -cat guardians make fewer hospital visits and are less likely to be on medication for heart problems and sleeping difficulties than non-guardians.
People with pet dogs took considerably more physical exercise than those with cats or those without pets; these effects are relatively long-term.
Pet guardianship has also been associated with increased survival in cases of coronary artery disease.
Human guardians are significantly less likely to die within one year of an acute myocardial infarction than those who do not own dogs.
Studies have found a small to moderate correlation between dog-ownership and increased adult physical-activity levels.
Health benefits of dogs can result from contact with dogs in general, not solely from having dogs as pets.
For example, when in a pet dog's presence, people show reductions in cardiovascular, behavioral, and psychological indicators of anxiety and are exposed to immune-stimulating microorganisms, which can protect against allergies and autoimmune diseases (according to the hygiene hypothesis).
Other benefits include dogs as social support.
One study indicated that wheelchair-users experience more positive social interactions with strangers when accompanied by a dog than when they are not.
In a 2015 study, it was found that having a pet made people more inclined to foster positive relationships with their neighbors.
In one study, new guardians reported a significant reduction in minor health problems during the first month following pet acquisition, which was sustained through the 10-month study.
Using dogs and other animals as a part of therapy dates back to the late-18th century, when animals were introduced into mental institutions to help socialize patients with mental disorders.
Animal-assisted intervention research has shown that animal-assisted therapy with a dog can increase smiling and laughing among people with Alzheimer's disease.
One study demonstrated that children with ADHD and conduct disorders who participated in an education program with dogs and other animals showed increased attendance, knowledge, and skill-objectives and decreased antisocial and violent behavior compared with those not in an animal-assisted program.
Artworks have depicted dogs as symbols of guidance, protection, loyalty, fidelity, faithfulness, alertness, and love.
In ancient Mesopotamia, from the Old Babylonian period until the Neo-Babylonian period, dogs were the symbol of Ninisina, the goddess of healing and medicine, and her worshippers frequently dedicated small models of seated dogs to her.
In the Neo-Assyrian and Neo-Babylonian periods, dogs served as emblems of magical protection.
In China, Korea, and Japan, dogs are viewed as kind protectors.
In mythology, dogs often appear as pets or as watchdogs.
Stories of dogs guarding the gates of the underworld recur throughout Indo-European mythologies and may originate from Proto-Indo-European traditions.
In Greek mythology, Cerberus is a three-headed, dragon-tailed watchdog who guards the gates of Hades.
Dogs also feature in association with the Greek goddess Hecate.
In Norse mythology, a dog called Garmr guards Hel, a realm of the dead.
In Persian mythology, two four-eyed dogs guard the Chinvat Bridge.
In Welsh mythology, Cŵn Annwn guards Annwn.
In Hindu mythology, Yama, the god of death, owns two watchdogs named Shyama and Sharvara, which each have four eyes—they are said to watch over the gates of Naraka.
A black dog is considered to be the vahana (vehicle) of Bhairava (an incarnation of Shiva).
In Christianity, dogs represent faithfulness.
Within the Roman Catholic denomination specifically, the iconography of Saint Dominic includes a dog after the saint's mother dreamt of a dog springing from her womb and became pregnant shortly after that.
As such, the Dominican Order (Ecclesiastical Latin: Domini canis) means "dog of the Lord" or "hound of the Lord".
In Christian folklore, a church grim often takes the form of a black dog to guard Christian churches and their churchyards from sacrilege.
Jewish law does not prohibit keeping dogs and other pets but requires Jews to feed dogs (and other animals that they own) before themselves and to make arrangements for feeding them before obtaining them.
The view on dogs in Islam is mixed, with some schools of thought viewing them as unclean, although Khaled Abou El Fadl states that this view is based on "pre-Islamic Arab mythology" and "a tradition [...] falsely attributed to the Prophet".
The Sunni Maliki school jurists disagree with the idea that dogs are unclean.
Dog – the species (or subspecies) as a whole, also any male member of the same.
Bitch – any female member of the species (or subspecies).
Puppy or pup – a young member of the species (or subspecies) under 12 months old.
Sire – the male parent of a litter.
Dam – the female parent of a litter.
Litter – all of the puppies resulting from a single whelping.
Whelping – the act of a bitch giving birth.
Whelps – puppies still dependent upon their dam.

Based on:
  https://en.wikipedia.org/wiki/Dog

*   The cat (Felis catus), also referred to as the domestic cat or house cat, is a small domesticated carnivorous mammal.
*   It is the only domesticated species of the family Felidae.
*   Domestication of the cat occurred in the Near East around 7500 BC.
*   It is commonly kept as a pet and working cat, but also ranges freely as a feral cat avoiding human contact.
*   It is valued by humans for companionship and its ability to kill vermin.
*   Its retractable claws are adapted to killing small prey species such as mice and rats.
*   It has a strong, flexible body, quick reflexes, and sharp teeth.
*   Its night vision and sense of smell are well developed.
*   It is a social species, but a solitary hunter and a crepuscular predator.
*   Cat intelligence is evident in their ability to adapt, learn through observation, and solve problems.
*   Research has shown they possess strong memories, exhibit neuroplasticity, and display cognitive skills comparable to those of a young child.
*   Cat communication includes meowing, purring, trilling, hissing, growling, grunting, and body language.
*   It can hear sounds too faint or too high in frequency for human ears, such as those made by small mammals.
*   It secretes and perceives pheromones.
*   Female domestic cats can have kittens from spring to late autumn in temperate zones and throughout the year in equatorial regions.
*   Litter sizes often range from two to five kittens.
*   Domestic cats are bred and shown at cat fancy events as registered pedigreed cats.
*   Population control includes spaying and neutering.
*   Pet abandonment has exploded the global feral cat population, which has driven the extinction of bird, mammal, and reptile species.
*   Domestic cats are found across the globe.
*   Out of the estimated 600 million cats worldwide, 400 million reside in Asia, including 58 million pet cats in China.
*   The United States leads in cat ownership with 73.8 million cats.
*   In the United Kingdom, approximately 10.9 million domestic cats are kept as pets.
*   The origin of the English word cat, Old English catt, is thought to be the Late Latin word cattus, first used at the beginning of the 6th century.
*   The Late Latin word may be derived from an unidentified African language (Nubian kaddîska, Nobiin kadīs are possible sources/cognates).
*   The forms might also have derived from an ancient Germanic word absorbed into Latin and then into Greek, Syriac, and Arabic.
*   The word may be derived from Germanic and Northern European languages, and ultimately be borrowed from Uralic (*käďwä, female (of a furred animal)).
*   The English puss, extended as pussy and pussycat, is attested from the 16th century and may have been introduced from Dutch poes or from Low German puuskatte.
*   Similar forms exist in Lithuanian puižė and Irish puisín or puiscín.
*   The etymology of "puss" is unknown, but it may be an onomatopoeia from using a sound to attract a cat.
*   A male cat is called a tom or tomcat (or a gib, if neutered).
*   A female is called a queen (or sometimes a molly, if spayed). Some sources write that queen refers solely to unspayed cats that are in an estrous cycle.
*   A juvenile cat is referred to as a kitten, a term interchangeable with the now-obsolete word catling in Early Modern English.
*   A group of cats can be referred to as a clowder, a glaring, or a colony.
*   The scientific name Felis catus was proposed by Carl Linnaeus in 1758 for a domestic cat.
*   Felis catus domesticus was proposed by Johann Christian Polycarp Erxleben in 1777.
*   Felis daemon proposed by Konstantin Satunin in 1904 was a black cat from the Transcaucasus, later identified as a domestic cat.
*   In 2003, the International Commission on Zoological Nomenclature ruled that the domestic cat is a distinct species, namely Felis catus.
*   In 2007, the modern domesticated subspecies F. silvestris catus sampled worldwide was considered to have probably descended from the African wildcat (F. lybica).
*   In 2017, the IUCN Cat Classification Taskforce followed the recommendation of the ICZN in regarding the domestic cat as a distinct species, Felis catus.
*   The domestic cat is a member of the Felidae, a family that has a common ancestor from about 10 to 15 million years ago.
*   The evolutionary radiation of the Felidae began in Asia during the Miocene around 8.38 to 14.45 million years ago.
*   Analysis of mitochondrial DNA of all Felidae species indicates a radiation at 6.46 to 16.76 million years ago.
*   The genus Felis genetically diverged from other Felidae around 6 to 7 million years ago.
*   Results of phylogenetic research shows that the wild members of this genus evolved through sympatric or parapatric speciation, whereas the domestic cat evolved through artificial selection.
*   The domestic cat and its closest wild ancestor are diploid and both possess 38 chromosomes and roughly 20,000 genes.
*   The earliest known indication for the taming of an African wildcat was excavated close by a human Neolithic grave in Shillourokambos, southern Cyprus, dating to about 7500–7200 BC.
*   African wildcats were attracted to early human settlements in the Fertile Crescent by rodents, in particular the house mouse (Mus musculus), and were tamed by Neolithic farmers.
*   The earliest known evidence for the occurrence of the domestic cat in Greece dates to around 1200 BC.
*   Greek, Phoenician, Carthaginian and Etruscan traders introduced domestic cats to southern Europe.
*   By the 5th century BC, they were familiar animals around settlements in Magna Graecia and Etruria.
*   During the Roman Empire, they were introduced to Corsica and Sardinia before the beginning of the 1st century AD.
*   By the end of the Western Roman Empire in the 5th century, the Egyptian domestic cat lineage had arrived in a Baltic Sea port in northern Germany.
*   The leopard cat (Prionailurus bengalensis) was tamed independently in China around 5500 BC, but this line leaves no trace in modern domestic cat populations.
*   During domestication, cats have undergone only minor changes in anatomy and behavior, and they are still capable of surviving in the wild.
*   Wildcat traits pre-adapted them for domestication: small size, social nature, obvious body language, love of play, and high intelligence.
*   Their rigorous grooming habits and instinct to bury their bodily waste make them generally much less messy than other domesticated animals.
*   House cats may mate with feral cats.
*   Hybridization between domestic and other Felinae species is also possible (e.g., Kellas cat).
*   Development of cat breeds started in the mid-19th century.
*   An analysis of the domestic cat genome revealed that the ancestral wildcat genome was significantly altered in the process of domestication, as specific mutations were selected to develop cat breeds.
*   Most breeds are founded on random-bred domestic cats.
*   Genetic diversity of these breeds varies between regions, and is lowest in purebred populations, which show more than 20 deleterious genetic disorders.
*   The domestic cat has a smaller skull and shorter bones than the European wildcat.
*   It averages about 46 cm (18 in) in head-to-body length and 23–25 cm (9.1–9.8 in) in height, with about 30 cm (12 in) long tails.
*   Males are larger than females.
*   Adult domestic cats typically weigh 4–5 kg (8.8–11.0 lb).
*   Cats have seven cervical vertebrae, 13 thoracic vertebrae, seven lumbar vertebrae, three sacral vertebrae, and a variable number of caudal vertebrae in the tail.
*   The extra lumbar and thoracic vertebrae account for the cat's spinal mobility and flexibility.
*   Attached to the spine are 13 ribs, the shoulder, and the pelvis.
*   Cat forelimbs are attached to the shoulder by free-floating clavicle bones.
*   The cat skull is unusual among mammals in having very large eye sockets and a powerful specialized jaw.
*   Two long canine teeth for killing and tearing prey, can stab between two of the prey's vertebrae and sever its spinal cord.
*   Domestic cats have narrowly spaced canine teeth relative to the size of their jaw, an adaptation to their preferred prey of small rodents.
*   The premolar and first molar together compose the carnassial pair on each side of the mouth, which efficiently shears meat.
*   Cats' small molars cannot chew food effectively, and cats are largely incapable of mastication.
*   Cats tend to have better dental health than most humans.
*   Cats have protractible and retractable claws.
*   Claws are sheathed in their normal, relaxed position, keeping them sharp and allowing silent stalking.
*   The claws on the forefeet are typically sharper than those on the hindfeet.
*   Cats can voluntarily extend their claws, such as in hunting, fighting, climbing, kneading, or for extra traction on soft surfaces.
*   Cats shed the outside layer of their claw sheaths when scratching rough surfaces.
*   Most cats have five claws on their front paws and four on their rear paws.
*   The dewclaw is proximal to the other claws.
*   A protrusion on the front paws inside the wrists appears to be a sixth "finger," thought to be an antiskidding device used while jumping.
*   Some cat breeds are prone to having extra digits ("polydactyly").
*   The cat is digitigrade, walking on its toes.
*   It uses a "pacing" gait that alternates both legs together on each side.
*   It registers directly by placing each hind paw close to the track of the corresponding fore paw, minimizing noise and visible tracks.
*   As it speeds up from walking to trotting, its gait changes to a "diagonal" gait.
*   Cats are generally fond of perching in high places.
*   A cat falling from up to 3 m (9.8 ft) can right itself and land on its paws.
*   During a lofty fall, a cat reflexively twists and rights itself to land on its feet using its acute sense of balance and flexibility; this is known as the cat righting reflex.
*   A cat always rights itself in the same way, and it has enough time in falls of at least 90 cm (3.0 ft).
*   The cat family (Felidae) can pass down many colors and patterns to their offspring.
*   The domestic cat genes MC1R and ASIP allow color variety in their coats.
*   Cats have excellent night vision and can see at one sixth the light level required for human vision.
*   Cat eyes have a tapetum lucidum, which reflects light back into the eye, increasing sensitivity to dim light.
*   Large pupils are an adaptation to dim light.
*   The domestic cat has slit pupils, which allow it to focus bright light without chromatic aberration.
*   At low light, a cat's pupils expand to cover most of the exposed surface of its eyes.
*   The domestic cat has rather poor color vision and only two types of cone cells (blue and yellowish green).
*   Its ability to distinguish between red and green is limited.
*   Cats have a nictitating membrane, allowing them to blink without hindering their vision.
*   The domestic cat's hearing is most acute in the range of 500 Hz to 32 kHz.
*   It can detect an extremely broad range of frequencies ranging from 55 Hz to 79 kHz.
*   It can hear a range of 10.5 octaves, compared to about 9 octaves for humans and dogs.
*   Its hearing sensitivity is enhanced by its large movable outer ears (pinnae).
*   It can detect ultrasound, including ultrasonic calls from rodent prey.
*   Cats have socio-spatial cognitive abilities to create mental maps of familiar people's locations based on hearing their voices.
*   Cats have an acute sense of smell, due to their well-developed olfactory bulb and a large surface of olfactory mucosa (about 5.8 cm2).
*   Cats and many other animals have a Jacobson's organ in their mouths that is used in flehmening to sense certain aromas in a way humans cannot.
*   Cats are sensitive to pheromones such as 3-mercapto-3-methylbutan-1-ol, which they use to communicate through urine spraying and marking with scent glands.
*   About 70–80% of cats are affected by nepetalactone (in catnip, silver vine, valerian).
*   Cats have about 470 taste buds, compared to more than 9,000 on the human tongue.
*   Domestic and wild cats share a taste receptor gene mutation that keeps their sweet taste buds from binding to sugary molecules, leaving them with no ability to taste sweetness.
*   They do have taste bud receptors specialized for acids, amino acids (constituents of protein), and bitter tastes.
*   Cats' taste buds possess the receptors needed to detect umami, detecting inosine monophosphate and histidine (particularly enriched in tuna).
*   Cats distinctly prefer food temperature around 38 °C (100 °F), similar to a fresh kill.
*   Some cats reject cold food, which would signal to the cat that the prey is long dead and therefore possibly toxic or decomposing.
*   To aid with navigation and sensation, cats have dozens of movable whiskers (vibrissae) over their body, especially their faces.
*   Whiskers provide information on the width of gaps and on the location of objects in the dark, by touching objects directly and by sensing air currents.
*   They trigger protective blink reflexes to protect the eyes from damage.
*   Outdoor cats are active both day and night, although they tend to be slightly more active at night.
*   Domestic cats spend the majority of their time in the vicinity of their homes, but they can range a radius of many hundreds of meters.
*   They establish territories that vary considerably in size, in one study ranging 7–28 ha (17–69 acres).
*   Being low-light predators, they are generally crepuscular (more active near dawn and dusk).
*   House cats' behavior is also influenced by human activity.
*   Cats conserve energy by sleeping more than most animals, especially as they grow older.
*   The daily duration of sleep varies, usually between 12 and 16 hours, with 13 to 14 being the average.
*   Some cats can sleep as much as 20 hours.
*   The term "cat nap" for a short rest refers to the cat's tendency to fall asleep (lightly) for a brief period.
*   Short periods of rapid eye movement sleep are often accompanied by muscle twitches, suggesting dreaming.
*   Behavioral and personality traits depend on a complex interplay between genetic and environmental factors.
*   The social behavior of the domestic cat ranges from widely dispersed individuals to feral cat colonies that gather around a food source, based on groups of co-operating females.
*   Within such groups, one cat is usually dominant over the others.
*   Each cat in a colony holds a distinct territory, with sexually active males having the largest territories (about 10 times larger than females' and may overlap with several females' territories).
*   These territories are marked by urine spraying, rubbing objects at head height with secretions from facial glands, and by defecation.
*   Between these territories are neutral areas where cats watch and greet one another without territorial conflicts.
*   Outside neutral areas, territory holders usually chase away stranger cats, initially by staring, hissing, and growling, and then by short and violent, noisy attacks.
*   Although cats do not have a social survival strategy or herd behavior, they always hunt alone.
*   Life in proximity to humans and other domestic animals has led to a symbiotic social adaptation in cats.
*   Cats may express great affection toward humans or other animals.
*   Ethologically, a cat's human keeper functions as a mother surrogate.
*   Adult cats live in a type of extended kittenhood, a form of behavioral neoteny.
*   Their high-pitched sounds may mimic the cries of a hungry human infant.
*   Some pet cats are poorly socialized and older cats show aggressiveness toward newly arrived kittens.
*   Redirected aggression is a common form of aggression in multiple cat households, where agitation towards one stimulus is directed at another cat, pet, or human.
*   Domestic cats' scent rubbing behavior toward humans or other cats is thought to be a feline means of social bonding.
*   Domestic cats use many vocalizations for communication, including purring, trilling, hissing, growling/snarling, grunting, and several different forms of meowing.
*   Their body language, including position of ears and tail, relaxation of the whole body, and kneading of the paws, are all indicators of mood.
*   The tail and ears are particularly important social signal mechanisms; a raised tail indicates a friendly greeting, and flattened ears indicate hostility.
*   Tail-raising also indicates the cat's position in the group's social hierarchy, with dominant individuals raising their tails less often than subordinate ones.
*   Feral cats are generally silent.
*   Nose-to-nose touching is also a common greeting and may be followed by social grooming.
*   Purring may have developed as an evolutionary advantage as a signaling mechanism of reassurance between mother cats and nursing kittens, and is thought to be a care-soliciting signal.
*   Post-nursing cats also often purr as a sign of contentment.
*   Purring has been recorded in a wide variety of circumstances, most of which involve physical contact between the cat and another, presumably trusted individual.
*   Some cats purr continuously when chronically ill or in apparent pain.
*   Purring is proposed to be generated via a series of sudden build-ups and releases of pressure as the glottis is opened and closed, causing the vocal folds to separate forcefully.
*   The laryngeal muscles in control of the glottis are thought to be driven by a neural oscillator which generates a cycle of contraction and release every 30–40 milliseconds (frequency of 33 to 25 Hz).
*   Domestic cats observed in rescue facilities have 276 morphologically distinct facial expressions based on 26 facial movements.
*   Facial expressions have helped researchers detect pain in cats, using the feline grimace scale's five criteria: ear position, orbital tightening, muzzle tension, whisker change, and head position.
*   Cats are known for spending considerable amounts of time licking their coats to keep them clean.
*   The cat's tongue has backward-facing spines about 0.5 millimeter long, called lingual papillae, which contain keratin making them rigid and act like a hairbrush.
*   Some cats, particularly long-haired cats, occasionally regurgitate sausage-shaped 2–3 cm (0.8–1.2 in) long hairballs of fur.
*   Hairballs can be prevented with remedies that ease elimination of the hair through the gut, and regular grooming.
*   Cat intelligence refers to a cat's ability to solve problems, adapt to its environment, learn new behaviors, and communicate its needs.
*   A cat's brain contains around 250 million neurons in the cerebral cortex.
*   Cats display neuroplasticity.
*   They have well-developed memory retaining information for a decade or longer.
*   These memories are often intertwined with emotions, allowing cats to recall both positive and negative experiences.
*   While they excel in observational learning and problem-solving, studies conclude that they struggle with understanding cause-and-effect relationships in the same way that humans do.
*   Cat intelligence study is mostly from consideration of the domesticated cat.
*   Living in urban environments has exposed them to challenges that require adaptive behaviors, contributing to cognitive development.
*   Selective breeding and genetic changes have further influenced their intelligence.
*   Kittens learn essential survival skills by observing their mothers, while adult cats refine their abilities through trial and error.
*   Domestic cats, especially young kittens, are known for their love of play.
*   This behavior mimics hunting and is important in helping kittens learn to stalk, capture, and kill prey.
*   Cats also engage in play fighting, both with each other and with humans.
*   Cats tend to play with toys more when they are hungry.
*   Cats prefer to play with objects that resemble prey, such as small furry toys that move rapidly.
*   They become habituated to a toy they have played with before.
*   String is often used as a toy, but if eaten, it can cause a medical emergency.
*   The shape and structure of cats' cheeks is insufficient to allow them to take in liquids using suction.
*   Lapping at a rate of four times a second, the cat touches the smooth tip of its tongue to the surface of the water, and quickly retracts it like a corkscrew, drawing water upward into their mouths.
*   Feral cats and free-fed house cats consume several small meals in a day.
*   They select food based on its temperature, smell, and texture; they dislike chilled foods and respond most strongly to moist foods rich in amino acids.
*   Cats reject novel flavors (neophobia) and learn quickly to avoid foods that have tasted unpleasant.
*   Most adult cats are lactose intolerant.
*   Some develop pica, eating or chewing on things such as wool, plastic, cables, paper, string, aluminum foil, or even coal.
*   Cats hunt small prey, primarily birds and rodents, and are often used as a form of pest control.
*   Other common small creatures, such as lizards and snakes, may also become prey.
*   Cats use two hunting strategies: stalking prey actively, or waiting in ambush.
*   Domestic cats are a major predator of wildlife in the United States, killing an estimated 1.3 to 4.0 billion birds and 6.3 to 22.3 billion mammals annually.
*   In the recovery of ringed robins and dunnocks in Britain, 31% of deaths were a result of cat predation.
*   The presentation of prey to human guardians is a poorly understood element of cat hunting behavior.
*   Domestic males are more likely to fight than females.
*   The most common reason for feral cat fighting is competition between two males to mate with a female, and most fights are won by the heavier male.
*   Another common reason for fighting in domestic cats is the difficulty of establishing territories within a small home.
*   Female cats also fight over territory or to defend their kittens.
*   Neutering decreases or eliminates fighting behavior in many cases, suggesting a link to sex hormones.
*   When cats become aggressive, they try to appear larger and more threatening by raising their fur, arching their backs, turning sideways, hissing, or spitting.
*   Often, the ears are pointed down and back to avoid damage and listen for changes.
*   Cats may also vocalize loudly and bare their teeth.
*   Fights usually consist of grappling, slapping the face and body with the forepaws, and bites.
*   Cats throw themselves to the ground in a defensive posture to rake their opponent's belly with their hind legs.
*   Serious damage from fights is rare, usually limited to infections from scratches and bites.
*   Bites are probably the main route of transmission of the feline immunodeficiency virus.
*   Sexually active males are usually involved in many fights and have battered faces.
*   Cats are willing to threaten animals larger than them to defend their territory, such as dogs and foxes.
*   The cat secretes and perceives pheromones.
*   Female cats, called queens, are polyestrous with several estrus cycles during a year, lasting usually 21 days.
*   They are usually ready to mate between early February and August in northern temperate zones and throughout the year in equatorial regions.
*   Several males, called tomcats, are attracted to a female in heat and fight over her.
*   A male cat's penis has a band of about 120–150 backward-pointing penile spines, which are about 1 mm (0.04 in) long, inducing ovulation upon withdrawal.
*   After mating, the female cleans her vulva thoroughly and attacks males attempting to mate shortly after.
*   Cats are superfecund; a female may mate with more than one male, resulting in different kittens in a litter having different fathers.
*   The morula forms 124 hours after conception.
*   At 148 hours, early blastocysts form.
*   At 10–12 days, implantation occurs.
*   The gestation of queens lasts between 64 and 67 days, with an average of 65 days.
*   A study of 2,300 free-ranging queens found they had one to six kittens per litter, with an average of three kittens.
*   They produced a mean of 1.4 litters per year, but a maximum of three litters in a year.
*   Of 169 kittens, 127 died before they were six months old due to trauma (dog attacks and road accidents).
*   The first litter is usually smaller than subsequent litters.
*   Kittens are weaned between six and seven weeks of age.
*   Queens normally reach sexual maturity at 5–10 months, and males at 5–7 months, varying by breed.
*   Kittens reach puberty at the age of 9–10 months.
*   Cats are ready to go to new homes at about 12 weeks of age, when they are ready to leave their mother.
*   They can be surgically sterilized (spayed or castrated) as early as seven weeks to limit unwanted reproduction.
*   This surgery also prevents undesirable sex-related behavior, such as aggression, territory marking (spraying urine) in males, and yowling (calling) in females.
*   Traditionally, this surgery was performed at around six to nine months of age, but is increasingly performed before puberty, at about three to six months.
*   In the United States, about 80% of household cats are neutered.
*   The average lifespan of pet cats has risen in recent decades: about 7 years in the early 1980s, 9.4 years in 1995, and an average of about 13 years as of 2014 and 2023.
*   Neutering increases life expectancy; castrated male cats live twice as long as intact males, while spayed female cats live 62% longer than intact females.
*   Neutering confers some health benefits, such as a greater life expectancy and a decreased incidence of reproductive neoplasia.
*   However, neutering decreases metabolism and increases food intake, both of which can cause obesity.
*   About 250 heritable genetic disorders have been identified in cats; many are similar to human inborn errors of metabolism.
*   The high level of similarity among the metabolism of mammals allows many feline diseases to be diagnosed using genetic tests developed for humans.
*   Cats can be used as animal models in the study of human diseases.
*   Diseases affecting domestic cats include acute infections, parasitic infestations, injuries, and chronic diseases such as kidney disease, thyroid disease, and arthritis.
*   Vaccinations are available for many infectious diseases, as are treatments to eliminate parasites.
*   The domestic cat is a cosmopolitan species and occurs across much of the world.
*   It is adaptable and now present on all continents except Antarctica, and on 118 of the 131 main groups of islands.
*   Due to its ability to thrive in almost any terrestrial habitat, it is among the world's most invasive species.
*   It lives on small islands with no human inhabitants.
*   Feral cats can live in forests, grasslands, tundra, coastal areas, agricultural land, scrublands, urban areas, and wetlands.
*   It can readily interbreed with the wildcat, posing a danger to the genetic distinctiveness of some wildcat populations (e.g., Scotland, Hungary, Iberian Peninsula).
*   Its introduction to places where no native felines are present also contributes to the decline of native species.
*   Feral cats are domestic cats that were born in or have reverted to a wild state.
*   They are unfamiliar with and wary of humans and roam freely in urban and rural areas.
*   Estimates of the United States feral population range from 25 to 60 million.
*   Feral cats may live alone, but most are in large colonies, occupying a specific territory and usually associated with a source of food.
*   Famous feral cat colonies are in Rome around the Colosseum and Forum Romanum.
*   On islands, birds can contribute as much as 60% of a cat's diet.
*   Domestic cats are a contributing factor to the decline of several species, leading to extinction in some cases (e.g., South Island piopio, Chatham rail, New Zealand merganser, Lyall's wren).
*   One feral cat in New Zealand killed 102 New Zealand lesser short-tailed bats in seven days.
*   In Australia, one study found feral cats to kill 466 million reptiles per year.
*   More than 258 reptile species were identified as being predated by cats.
*   Cats have contributed to the extinction of the Navassa curly-tailed lizard and Chioninia coctei.
*   Cats are common pets throughout the world, and their worldwide population as of 2007 exceeded 500 million.
*   As of 2024, the domestic cat was the second most popular pet in the United States, with 73.8 million cats owned and around 42.2 million households owning at least one cat.
*   In the United Kingdom, 26% of adults have a cat, with an estimated population of 10.9 million pet cats as of 2020.
*   As of 2021, there were an estimated 220 million owned and 480 million stray cats in the world.
*   Cats have been used for millennia to control rodents, notably around grain stores and aboard ships.
*   Cats are also used in the international fur trade and leather industries for making coats, hats, blankets, stuffed toys, shoes, gloves, and musical instruments.
*   About 24 cats are needed to make a cat-fur coat.
*   This use has been outlawed in the United States since 2000 and in the European Union (as well as the United Kingdom) since 2007.
*   Cat pelts have been used for superstitious purposes as part of the practice of witchcraft, and are still made into blankets in Switzerland as traditional medicines.
*   General estimates for the global population of domestic cats range widely from anywhere between 200 million to 600 million.
*   Walter Chandoha made his career photographing cats after his 1949 images of Loco, a stray cat, were published.
*   He is reported to have photographed 90,000 cats during his career and maintained an archive of 225,000 images.
*   Pet humanization is a form of anthropomorphism in which cats are kept for companionship and treated more like human family members.
*   A cat show is a judged event in which the owners of cats compete to win titles in various cat-registering organizations by entering their cats to be judged after a breed standard.
*   It is often required that a cat must be healthy and vaccinated to participate in a cat show.
*   Both pedigreed and non-purebred companion ("moggy") cats are admissible.
*   Competing cats are compared to the applicable breed standard, and assessed for temperament.
*   Cats can be infected or infested with viruses, bacteria, fungus, protozoans, arthropods, or worms that can transmit diseases to humans.
*   The likelihood that a person will become diseased depends on the age and immune status of the person.
*   Humans who have cats living in their home or in close association are more likely to become infected.
*   Others might also acquire infections from cat feces and parasites exiting the cat's body.
*   Some of the infections of most concern include salmonella, cat-scratch disease, and toxoplasmosis.
*   In ancient Egypt, cats were revered, and the goddess Bastet often depicted in cat form.
*   The Greek historian Herodotus reported that killing a cat was forbidden, and when a household cat died, the entire family mourned and shaved their eyebrows.
*   Families took their dead cats to the sacred city of Bubastis, where they were embalmed and buried.
*   Ancient Greeks and Romans kept weasels as pets, which were seen as the ideal rodent-killers.
*   The earliest unmistakable evidence of the Greeks having domestic cats comes from two coins from Magna Graecia dating to the mid-fifth century BC.
*   The usual ancient Greek word for 'cat' was ailouros, meaning 'thing with the waving tail'.
*   Cats are rarely mentioned in ancient Greek literature.
*   Aristotle remarked in his History of Animals that "female cats are naturally lecherous".
*   The Greeks later syncretized their own goddess Artemis with the Egyptian goddess Bastet, adopting Bastet's associations with cats.
*   In Ovid's Metamorphoses, when the deities flee to Egypt and take animal forms, the goddess Diana turns into a cat.
*   Cats eventually displaced weasels as the pest control of choice.
*   During the Middle Ages, many of Artemis's associations with cats were grafted onto the Virgin Mary.
*   Cats are often shown in icons of Annunciation and of the Holy Family.
*   According to Italian folklore, on the same night that Mary gave birth to Jesus, a cat in Bethlehem gave birth to a kitten.
*   Domestic cats were spread throughout much of the rest of the world during the Age of Discovery, as ships' cats were carried on sailing ships to control shipboard rodents and as good-luck charms.
*   Several ancient religions believed cats are exalted souls, companions or guides for humans, that are all-knowing but mute.
*   In Japan, the maneki neko cat is a symbol of good fortune.
*   In Norse mythology, Freyja, the goddess of love, beauty, and fertility, is depicted as riding a chariot drawn by cats.
*   In Jewish legend, the first cat was living in the house of the first man Adam as a pet that got rid of mice.
*   Although no species are sacred in Islam, cats are revered by Muslims.
*   One of the companions of Muhammad was known as Abu Hurayrah ("father of the kitten"), in reference to his documented affection to cats.
*   Many cultures have negative superstitions about cats.
*   For example, that encountering a black cat ("crossing one's path") leads to bad luck, or that cats are witches' familiar spirits.
*   The killing of cats in medieval Ypres, Belgium, is commemorated in the present-day Kattenstoet (cat parade).
*   In mid-16th century France, cats were allegedly burnt alive as a form of entertainment, particularly during midsummer festivals.
*   According to a myth in many cultures, cats have multiple lives.
*   In many countries, they are believed to have nine lives, but in Italy, Germany, Greece, Brazil, and some Spanish-speaking regions, they are said to have seven lives, while in Arabic traditions, the number of lives is six.
*   An early mention of the myth is in John Heywood's The Proverbs of John Heywood (1546).
*   The myth is attributed to the natural suppleness and swiftness cats exhibit to escape life-threatening situations.
*   Falling cats often land on their feet, using an instinctive righting reflex.

Based on:
  https://en.wikipedia.org/wiki/Cat

Here's the relevant information about "cat" from the source:

*   Schrödinger's cat is a hypothetical cat in a closed box.
*   The cat may be considered to be simultaneously both alive and dead while it is unobserved.
*   Its fate is linked to a random subatomic event that may or may not occur.
*   In Schrödinger's original formulation, a cat, a flask of poison, and a radioactive source are placed in a sealed box.
*   If an internal radiation monitor detects radioactivity, the flask is shattered, releasing the poison, which kills the cat.
*   If no decaying atom triggers the monitor, the cat remains alive.
*   The Copenhagen interpretation implies that the cat is therefore simultaneously alive and dead.
*   When one looks in the box, one sees the cat either alive or dead, not both.
*   The experiment is not intended to be actually performed on a cat.
*   Superposing an object as large as a cat would pose considerable technical difficulties.
*   The Schrödinger's cat experiment asks how long quantum superpositions last and when (or whether) they collapse, referring to the cat's state.
*   Schrödinger proposed a scenario with a cat in a closed steel chamber, wherein the cat's life or death depended on the state of a radioactive atom.
*   According to Schrödinger, the Copenhagen interpretation implies that the cat remains both alive and dead until the state has been observed.
*   Schrödinger did not wish to promote the idea of dead-and-live cats as a serious possibility; he intended the example to illustrate the absurdity of the existing view of quantum mechanics.
*   The wave function of the entire system would express this by having in it the living and dead cat mixed or spread out in equal parts.
*   The reality of the cat mixes the living and dead cat.
*   Nobody really doubts that the presence or absence of the cat is something independent of the act of observation.
*   Schrodinger's hypothetical cat experiment describes the measurement problem: quantum theory describes the cat system as a combination of two possible outcomes but only one outcome is ever observed.
*   Our intuition says that the cat cannot be in more than one state simultaneously.
*   In the Copenhagen interpretation, the wavefunction description of the system consists of a superposition of the states "decayed nucleus/dead cat" and "undecayed nucleus/living cat".
*   Only when the box is opened and observed can we make a statement about the cat.
*   Eugene Wigner asserted that an observer is necessary for a collapse to one or the other (e.g., either a live cat or a dead cat) of the terms on the right-hand side of a wave function.
*   Wigner supposed that a friend opened the box and observed the cat without telling anyone.
*   A resolution of the paradox is that the triggering of the Geiger counter counts as a measurement of the state of the radioactive substance, deciding the state of the cat.
*   Bohr viewed the state of the cat before the box is opened as indeterminate.
*   Schrödinger's cat would be either dead or alive long before the box is opened but the cat and box form an inseparable combination.
*   In the many-worlds interpretation, both alive and dead states of the cat persist after the box is opened.
*   When the box is opened, the observer and the possibly-dead cat split into an observer looking at a box with a dead cat and an observer looking at a box with a live cat.
*   When opening the box, the observer becomes entangled with the cat.
*   A variant of the Schrödinger's cat experiment, known as the quantum suicide machine, has been proposed by cosmologist Max Tegmark. It examines the Schrödinger's cat experiment from the point of view of the cat.
*   The ensemble interpretation states that the state vector would not apply to individual cat experiments, but only to the statistics of many similarly prepared cat experiments.
*   In the relational interpretation, the cat can be considered an observer of the apparatus.
*   To the cat, the wavefunction of the apparatus has appeared to "collapse".
*   In the transactional interpretation, the cat is never in superposition. Rather the cat is only in one state at any particular time.
*   According to objective collapse theories, the cat would be expected to have settled into a definite state long before the box is opened.
*   "The cat observes itself" or "the environment observes the cat".
*   Experiments involving similar principles, e.g. superpositions of relatively large objects, have been performed, pushing the upper limit on "cat states".
*   In quantum computing the phrase "cat state" sometimes refers to the GHZ state, wherein several qubits are in an equal superposition of all being 0 and all being 1.
*   It may be possible to determine the state of the cat before observing it.
*   Studies involving interpretations of quantum mechanics are related to the Schrödinger's cat paradox.

Based on:
  https://en.wikipedia.org/wiki/Schr%C3%B6dinger%27s_cat

Now send the `summaries` to the model.

In [18]:
# Create a function response part
function_response_part = types.Part.from_function_response(
    name=fc.name,
    response={"result": summaries},
)

response = chat.send_message(function_response_part)

display(Markdown(response.text))

Dogs and cats, though both domesticated carnivorous mammals, exhibit significant differences in their characteristics.

**Domestication and Ancestry:**
*   **Dogs:** Descended from the gray wolf, dogs were domesticated over 14,000 years ago by hunter-gatherers, even before the development of agriculture. They are the first species to be domesticated by humans.
*   **Cats:** Domesticated in the Near East around 7500 BC, the domestic cat is the only domesticated species of the family Felidae. Their wild ancestor is thought to be the African wildcat.

**Physical Characteristics:**
*   **Dogs:** Show wide variation in shape, size, and color across breeds. They have powerful jaws with around 42 teeth and typically have 26 vertebrae in their tails, though some breeds have naturally short tails. Their skeletons are well-adapted for running.
*   **Cats:** Average about 46 cm (18 in) in head-to-body length and 23–25 cm (9.1–9.8 in) in height. They have retractable claws, very large eye sockets, and specialized jaws with two long canine teeth for killing prey. Most cats have five claws on their front paws and four on their rear paws.

**Senses:**
*   **Dogs:** Possess a superior sense of smell and hearing compared to humans, but inferior visual acuity. Their vision is dichromatic, meaning they see in yellows, blues, and grays, with difficulty differentiating between red and green. They can detect sounds from about 400m.
*   **Cats:** Have excellent night vision, requiring only one-sixth the light level humans need. Their hearing is extremely acute, ranging from 55 Hz to 79 kHz, and they can detect ultrasound. They have an acute sense of smell due to a well-developed olfactory bulb. Cats have about 470 taste buds and cannot taste sweetness.

**Diet and Hunting:**
*   **Dogs:** Are typically described as omnivores and have adapted to a starch-rich diet, unlike other canids. They use their carnassial teeth to shear meat.
*   **Cats:** Are carnivorous mammals and are solitary hunters, primarily preying on birds and rodents. Their narrowly spaced canine teeth are an adaptation for their preferred small rodent prey. Cats are largely incapable of mastication.

**Behavior and Intelligence:**
*   **Dogs:** Have uniquely adapted to human behavior, understanding and communicating with humans. They have about twice the number of neurons in their cerebral cortexes compared to cats, suggesting higher intelligence. They can be trained for various roles like hunting, herding, and assistance.
*   **Cats:** Exhibit intelligence through their ability to adapt, learn through observation, and solve problems. They possess strong memories and display cognitive skills comparable to a young child. They are generally crepuscular predators.

**Communication:**
*   **Dogs:** Communicate through eye gaze, facial expressions, vocalizations, body posture, and gustatory communication (scents, pheromones, and taste). They mark territories by urinating.
*   **Cats:** Use various vocalizations like meowing, purring, trilling, hissing, growling, and body language, including ear and tail positions. They secrete and perceive pheromones and use urine spraying and rubbing to mark territories.

**Lifespan and Health:**
*   **Dogs:** Median longevity is approximately 12.7 years, with purebred dogs generally living longer than crossbred dogs. Obesity negatively correlates with longevity. They are susceptible to numerous disorders, and are a significant vector for rabies.
*   **Cats:** Average lifespan for pet cats is about 13 years. Neutering increases life expectancy and reduces the incidence of reproductive neoplasia. They can be infected with various pathogens and parasites, some of which can transmit diseases to humans.

**Social Aspects:**
*   **Dogs:** Are highly social and form strong bonds with humans, earning the sobriquet "man's best friend." They are the most popular pet in the United States.
*   **Cats:** Can be social but are solitary hunters. They may form colonies around food sources and show affection towards humans. Pet humanization is common, with cats treated as family members.

This information provides a general overview, and specific characteristics can vary greatly between individual animals and breeds.

## Re-ranking the search results

Helper function to embed the content:

In [19]:
# @title Helper function to embed the content
from tqdm.auto import tqdm
from google.genai import types

tqdm.pandas()

from google.api_core import retry
import numpy as np

def make_embed_text_fn(model):

    @retry.Retry(timeout=300.0)
    def embed_fn(texts: list[str]) -> list[list[float]]:
        # Set the task_type to SEMANTIC_SIMILARITY and embed the batch of texts
        embeddings = client.models.embed_content(
            model=model,
            contents=texts,
            config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY"),
        ).embeddings
        return np.array([embedding.values for embedding in embeddings])

    return embed_fn


def create_embeddings(content: list[str]) -> np.ndarray:
    MODEL_ID = "embedding-001" # @param ["embedding-001", "text-embedding-004","gemini-embedding-exp-03-07"] {"allow-input":true, isTemplate: true}
    model = f"models/{MODEL_ID}"
    embed_fn = make_embed_text_fn(model)

    batch_size = 100  # at most 100 requests can be in one batch
    all_embeddings = []

    # Loop over the texts in chunks of batch_size
    for i in tqdm(range(0, len(content), batch_size)):
        batch = content[i:i + batch_size]
        embeddings = embed_fn(batch)
        all_embeddings.extend(embeddings)

    return np.array(all_embeddings).reshape(len(all_embeddings), -1)

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and so their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [20]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [21]:
search_res = create_embeddings(summaries)
embedded_query = create_embeddings([query])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity score:

In [22]:
sim_value = dot_product(search_res, embedded_query)
sim_value

array([[0.66220313],
       [0.65549758],
       [0.61322208]])

using `np.argmax` best candidate is selected.

**User's Input:** Compare dog and cat characteristics.

**Answer:**

In [23]:
display(Markdown(summaries[np.argmax(sim_value)]))

The dog (Canis familiaris or Canis lupus familiaris) is a domesticated descendant of the gray wolf.
Also called the domestic dog, it was selectively bred from an extinct population of wolves during the Late Pleistocene by hunter-gatherers.
The dog was the first species to be domesticated by humans, over 14,000 years ago and before the development of agriculture.
Due to their long association with humans, dogs have gained the ability to thrive on a starch-rich diet that would be inadequate for other canids.
Dogs have been bred for desired behaviors, sensory capabilities, and physical attributes.
Dog breeds vary widely in shape, size, and color.
They have the same number of bones (with the exception of the tail), powerful jaws that house around 42 teeth, and well-developed senses of smell, hearing, and sight.
Compared to humans, dogs possess a superior sense of smell and hearing, but inferior visual acuity.
Dogs perform many roles for humans, such as hunting, herding, pulling loads, protection, companionship, therapy, aiding disabled people, and assisting police and the military.
Communication in dogs includes eye gaze, facial expression, vocalization, body posture (including movements of bodies and limbs), and gustatory communication (scents, pheromones, and taste).
They mark their territories by urinating on them, which is more likely when entering a new environment.
Over the millennia, dogs have uniquely adapted to human behavior; this adaptation includes being able to understand and communicate with humans.
As such, the human–canine bond has been a topic of frequent study, and dogs' influence on human society has given them the sobriquet of "man's best friend".
The global dog population is estimated at 700 million to 1 billion, distributed around the world.
The dog is the most popular pet in the United States, present in 34–40% of households.
Developed countries make up approximately 20% of the global dog population, while around 75% of dogs are estimated to be from developing countries, mainly in the form of feral and community dogs.
Dogs are domesticated members of the family Canidae.
They are classified as a subspecies of Canis lupus, along with wolves and dingoes.
Dogs were domesticated from wolves over 14,000 years ago by hunter-gatherers, before the development of agriculture.
The remains of the Bonn–Oberkassel dog, buried alongside humans between 14,000 and 15,000 years ago, are the earliest to be conclusively identified as a domesticated dog.
Genetic studies show that dogs likely diverged from wolves between 27,000 and 40,000 years ago.
The dingo and the related New Guinea singing dog resulted from the geographic isolation and feralization of dogs in Oceania over 8,000 years ago.
Dogs, wolves, and dingoes have sometimes been classified as separate species.
In 1758, the Swedish botanist and zoologist Carl Linnaeus assigned the genus name Canis (which is the Latin word for "dog") to the domestic dog, the wolf, and the golden jackal in his book, Systema Naturae.
He classified the domestic dog as Canis familiaris.
Linnaeus considered the dog to be a separate species from the wolf because of its upturning tail (cauda recurvata in Latin term), which is not found in any other canid.
In the 2005 edition of Mammal Species of the World, mammalogist W. Christopher Wozencraft listed the wolf as a wild subspecies of Canis lupus and proposed two additional subspecies: familiaris, as named by Linnaeus in 1758, and dingo, named by Meyer in 1793.
Wozencraft included hallstromi (the New Guinea singing dog) as another name (junior synonym) for the dingo.
The earliest remains generally accepted to be those of a domesticated dog were discovered in Bonn-Oberkassel, Germany.
Contextual, isotopic, genetic, and morphological evidence shows that this dog was not a local wolf.
The dog was dated to 14,223 years ago and was found buried along with a man and a woman, all three having been sprayed with red hematite powder and buried under large, thick basalt blocks.
The dog had died of canine distemper.
This timing indicates that the dog was the first species to be domesticated in the time of hunter-gatherers, which predates agriculture.
DNA sequences show that all ancient and modern dogs share a common ancestry and descended from an ancient, extinct wolf population that was distinct from any modern wolf lineage.
The dog is a domestic animal that likely travelled a commensal pathway into domestication (i.e. humans initially neither benefitted nor were harmed by wild dogs eating refuse from their camps).
In 2021, a literature review of the current evidence infers that the dog was domesticated in Siberia 23,000 years ago by ancient North Siberians, then later dispersed eastward into the Americas and westward across Eurasia, with dogs likely accompanying the first humans to inhabit the Americas.
In 2018, a study identified 429 genes that differed between modern dogs and modern wolves.
As the differences in these genes could also be found in ancient dog fossils, these were regarded as being the result of the initial domestication and not from recent breed formation.
These genes are linked to neural crest and central nervous system development.
These genes affect embryogenesis and can confer tameness, smaller jaws, floppy ears, and diminished craniofacial development, which distinguish domesticated dogs from wolves and are considered to reflect domestication syndrome.
The study concluded that during early dog domestication, the initial selection was for behavior.
This trait is influenced by those genes which act in the neural crest, which led to the phenotypes observed in modern dogs.
There are around 450 official dog breeds, the most of any mammal.
Dogs began diversifying in the Victorian era, when humans took control of their natural selection.
Most breeds were derived from small numbers of founders within the last 200 years.
Since then, dogs have undergone rapid phenotypic change and have been subjected to artificial selection by humans.
The skull, body, and limb proportions between breeds display more phenotypic diversity than can be found within the entire order of carnivores.
These breeds possess distinct traits related to morphology, which include body size, skull shape, tail phenotype, fur type, and colour.
As such, humans have long used dogs for their desirable traits to complete or fulfill a certain work or role.
Their behavioural traits include guarding, herding, hunting, retrieving, and scent detection.
Their personality traits include hypersocial behavior, boldness, and aggression.
Present-day dogs are dispersed around the world.
Dogs are extremely variable in size, ranging from one of the largest breeds, the Great Dane, at 50 to 79 kg (110 to 174 lb) and 71 to 81 cm (28 to 32 in), to one of the smallest, the Chihuahua, at 0.5 to 3 kg (1.1 to 6.6 lb) and 13 to 20 cm (5.1 to 7.9 in).
All healthy dogs, regardless of their size and type, have the same amount of bones (with the exception of the tail), although there is significant skeletal variation between dogs of different types.
The dog's skeleton is well adapted for running; the vertebrae on the neck and back have extensions for back muscles, consisting of epaxial muscles and hypaxial muscles, to connect to; the long ribs provide room for the heart and lungs; and the shoulders are unattached to the skeleton, allowing for flexibility.
Compared to the dog's wolf-like ancestors, selective breeding since domestication has seen the dog's skeleton increase in size for larger types such as mastiffs and miniaturised for smaller types such as terriers.
Most dogs naturally have 26 vertebrae in their tails, but some with naturally short tails have as few as three.
The dog's skull has identical components regardless of breed type, but there is significant divergence in terms of skull shape between types.
The three basic skull shapes are the elongated dolichocephalic type as seen in sighthounds, the intermediate mesocephalic or mesaticephalic type, and the very short and broad brachycephalic type exemplified by mastiff type skulls.
The jaw contains around 42 teeth, and it has evolved for the consumption of flesh.
Dogs use their carnassial teeth to cut food into bite-sized chunks, more especially meat.
Dogs' senses include vision, hearing, smell, taste, touch, and magnetoreception.
One study suggests that dogs can feel small variations in Earth's magnetic field.
Dogs prefer to defecate with their spines aligned in a north–south position in calm magnetic field conditions.
Dogs' vision is dichromatic; their visual world consists of yellows, blues, and grays.
They have difficulty differentiating between red and green, and much like other mammals, the dog's eye is composed of two types of cone cells compared to the human's three.
The divergence of the eye axis of dogs ranges from 12 to 25°, depending on the breed, which can have different retina configurations.
The fovea centralis area of the eye is attached to a nerve fiber, and is the most sensitive to photons.
Additionally, a study found that dogs' visual acuity was up to eight times less effective than a human, and their ability to discriminate levels of brightness was about two times worse than a human.
While the human brain is dominated by a large visual cortex, the dog brain is dominated by a large olfactory cortex.
Dogs have roughly forty times more smell-sensitive receptors than humans, ranging from about 125 million to nearly 300 million in some dog breeds, such as bloodhounds.
This sense of smell is the most prominent sense of the species; it detects chemical changes in the environment, allowing dogs to pinpoint the location of mating partners, potential stressors, resources, etc.
Dogs also have an acute sense of hearing up to four times greater than that of humans.
They can pick up the slightest sounds from about 400 m (1,300 ft) compared to 90 m (300 ft) for humans.
Dogs have stiff, deeply embedded hairs known as whiskers that sense atmospheric changes, vibrations, and objects not visible in low light conditions.
The lower most part of whiskers hold more receptor cells than other hair types, which help in alerting dogs of objects that could collide with the nose, ears, and jaw.
Whiskers likely also facilitate the movement of food towards the mouth.
The coats of domestic dogs are of two varieties: "double" being common in dogs (as well as wolves) originating from colder climates, made up of a coarse guard hair and a soft down hair, or "single", with the topcoat only.
Breeds may have an occasional "blaze", stripe, or "star" of white fur on their chest or underside.
Premature graying can occur in dogs as early as one year of age; this is associated with impulsive behaviors, anxiety behaviors, and fear of unfamiliar noise, people, or animals.
Some dog breeds are hairless, while others have a very thick corded coat.
A dog's dewclaw is the fifth digit in its forelimb and hind legs.
Dewclaws on the forelimbs are attached by bone and ligament, while the dewclaws on the hind legs are attached only by skin.
Most dogs are not born with dewclaws in their hind legs, and some are without them in their forelimbs.
Dogs' dewclaws consist of the proximal phalanges and distal phalanges.
A dog's tail is the terminal appendage of the vertebral column, which is made up of a string of 5 to 23 vertebrae enclosed in muscles and skin that support the dog's back extensor muscles.
One of the primary functions of a dog's tail is to communicate their emotional state.
The tail also helps the dog maintain balance by putting its weight on the opposite side of the dog's tilt, and it can also help the dog spread its anal gland's scent through the tail's position and movement.
Dogs can have a violet gland (or supracaudal gland) characterized by sebaceous glands on the dorsal surface of their tails; in some breeds, it may be vestigial or absent.
A study suggests that dogs show asymmetric tail-wagging responses to different emotive stimuli.
Dogs can injure themselves by wagging their tails forcefully; this condition is called kennel tail, happy tail, bleeding tail, or splitting tail.
In some hunting dogs, the tail is traditionally docked to avoid injuries.
Some dogs can be born without tails because of a DNA variant in the T gene, which can also result in a congenitally short (bobtail) tail.
Numerous disorders have been known to affect dogs.
Some are congenital and others are acquired.
Dogs can acquire upper respiratory tract diseases including diseases that affect the nasal cavity, the larynx, and the trachea; lower respiratory tract diseases which includes pulmonary disease and acute respiratory diseases; heart diseases which includes any cardiovascular inflammation or dysfunction of the heart; haemopoietic diseases including anaemia and clotting disorders; gastrointestinal disease such as diarrhoea and gastric dilatation volvulus; hepatic disease such as portosystemic shunts and liver failure; pancreatic disease such as pancreatitis; renal disease; lower urinary tract disease such as cystitis and urolithiasis; endocrine disorders such as diabetes mellitus, Cushing's syndrome, hypoadrenocorticism, and hypothyroidism; nervous system diseases such as seizures and spinal injury; musculoskeletal disease such as arthritis and myopathies; dermatological disorders such as alopecia and pyoderma; ophthalmological diseases such as conjunctivitis, glaucoma, entropion, and progressive retinal atrophy; and neoplasia.
Common dog parasites are lice, fleas, fly larvae, ticks, mites, cestodes, nematodes, and coccidia.
Taenia is a notable genus with 5 species in which dogs are the definitive host.
Additionally, dogs are a source of zoonoses for humans.
They are responsible for 99% of rabies cases worldwide.
Other common zoonoses are hydatid disease, leptospirosis, pasteurellosis, ringworm, and toxocariasis.
Common infections in dogs include canine adenovirus, canine distemper virus, canine parvovirus, leptospirosis, canine influenza, and canine coronavirus.
All of these conditions have vaccines available.
Dogs are the companion animal most frequently reported for exposure to toxins.
Most poisonings are accidental and over 80% of reports of exposure to the ASPCA animal poisoning hotline are due to oral exposure.
Neoplasia is the most common cause of death for dogs.
Other common causes of death are heart and renal failure.
Their pathology is similar to that of humans, as is their response to treatment and their outcomes.
Genes found in humans to be responsible for disorders are investigated in dogs as being the cause and vice versa.
The typical lifespan of dogs varies widely among breeds, but the median longevity (the age at which half the dogs in a population have died and half are still alive) is approximately 12.7 years.
Obesity correlates negatively with longevity with one study finding obese dogs to have a life expectancy approximately a year and a half less than dogs with a healthy weight.
In a 2024 UK study analyzing 584,734 dogs, it was concluded that purebred dogs lived longer than crossbred dogs, challenging the previous notion of the latter having the higher life expectancies.
Other studies also show that fully mongrel dogs live about a year longer on average than dogs with pedigrees.
Furthermore, small dogs with longer muzzles have been shown to have higher lifespans than larger medium-sized dogs with much more depressed muzzles.
For free-ranging dogs, less than 1 in 5 reach sexual maturity, and the median life expectancy for feral dogs is less than half of dogs living with humans.
In domestic dogs, sexual maturity happens around six months to one year for both males and females, although this can be delayed until up to two years of age for some large breeds.
This is the time at which female dogs will have their first estrous cycle, characterized by their vulvas swelling and producing discharges, usually lasting between 4 and 20 days.
They will experience subsequent estrous cycles semiannually, during which the body prepares for pregnancy.
At the peak of the cycle, females will become estrous, mentally and physically receptive to copulation.
Because the ova survive and can be fertilized for a week after ovulation, more than one male can sire the same litter.
Fertilization typically occurs two to five days after ovulation.
After ejaculation, the dogs are coitally tied for around 5–30 minutes because of the male's bulbus glandis swelling and the female's constrictor vestibuli contracting; the male will continue ejaculating until they untie naturally due to muscle relaxation.
14–16 days after ovulation, the embryo attaches to the uterus, and after seven to eight more days, a heartbeat is detectable.
Dogs bear their litters roughly 58 to 68 days after fertilization, with an average of 63 days, although the length of gestation can vary.
An average litter consists of about six puppies.
Neutering is the sterilization of animals via gonadectomy, which is an orchidectomy (castration) in dogs and ovariohysterectomy (spay) in bitches.
Neutering reduces problems caused by hypersexuality, especially in male dogs.
Spayed females are less likely to develop cancers affecting the mammary glands, ovaries, and other reproductive organs.
However, neutering increases the risk of urinary incontinence in bitches, prostate cancer in dogs, and osteosarcoma, hemangiosarcoma, cruciate ligament rupture, pyometra, obesity, and diabetes mellitus in either sex.
Neutering is the most common surgical procedure in dogs less than a year old in the US and is seen as a control method for overpopulation.
Neutering often occurs as early as 6–14 weeks in shelters in the US.
The American Society for the Prevention of Cruelty to Animals (ASPCA) advises that dogs not intended for further breeding should be neutered so that they do not have undesired puppies that may later be euthanized.
Neutering is less common in most European countries, especially in Nordic countries—except for the UK, where it is common.
In a study of seven dog breeds (the Bernese Mountain Dog, Basset Hound, Cairn Terrier, Brittany, German Shepherd Dog, Leonberger, and West Highland White Terrier), it was found that inbreeding decreases litter size and survival.
Another analysis of data on 42,855 Dachshund litters found that as the inbreeding coefficient increased, litter size decreased and the percentage of stillborn puppies increased, thus indicating inbreeding depression.
In a study of Boxer litters, 22% of puppies died before reaching 7 weeks of age.
Stillbirth was the most frequent cause of death, followed by infection.
Mortality due to infection increased significantly with increases in inbreeding.
Dog behavior has been shaped by millennia of contact with humans.
They have acquired the ability to understand and communicate with humans and are uniquely attuned to human behaviors.
Behavioral scientists suggest that a set of social-cognitive abilities in domestic dogs that are not possessed by the dog's canine relatives or other highly intelligent mammals, such as great apes, are parallel to children's social-cognitive skills.
Dogs have about twice the number of neurons in their cerebral cortexes than what cats have, which suggests they could be about twice as intelligent.
Most domestic animals were initially bred for the production of goods.
Dogs, on the other hand, were selectively bred for desirable behavioral traits.
In 2016, a study found that only 11 fixed genes showed variation between wolves and dogs.
These gene variations indicate the occurrence of artificial selection and the subsequent divergence of behavior and anatomical features.
These genes have been shown to affect the catecholamine synthesis pathway, with the majority of the genes affecting the fight-or-flight response (i.e., selection for tameness) and emotional processing.
Compared to their wolf counterparts, dogs tend to be less timid and less aggressive, though some of these genes have been associated with aggression in certain dog breeds.
Traits of high sociability and lack of fear in dogs may include genetic modifications related to Williams-Beuren syndrome in humans, which cause hypersociability at the expense of problem-solving ability.
In a 2023 study of 58 dogs, some dogs classified as attention deficit hyperactivity disorder-like showed lower serotonin and dopamine concentrations.
A similar study claims that hyperactivity is more common in male and young dogs.
A dog can become aggressive because of trauma or abuse, fear or anxiety, territorial protection, or protecting an item it considers valuable.
Acute stress reactions from post-traumatic stress disorder (PTSD) seen in dogs can evolve into chronic stress.
Police dogs with PTSD can often refuse to work.
Dogs have a natural instinct called prey drive (the term is chiefly used to describe training dogs' habits) which can be influenced by breeding.
These instincts can drive dogs to consider objects or other animals to be prey or drive possessive behavior.
These traits have been enhanced in some breeds so that they may be used to hunt and kill vermin or other pests.
Puppies or dogs sometimes bury food underground.
Some puppies and dogs engage in coprophagy out of habit, stress, for attention, or boredom; most of them will not do it later in life.
A study hypothesizes that the behavior was inherited from wolves, a behavior likely evolved to lessen the presence of intestinal parasites in dens.
Most dogs can swim.
In a study of 412 dogs, around 36.5% of the dogs could not swim; the other 63.5% were able to swim without a trainer in a swimming pool.
A study of 55 dogs found a correlation between swimming and 'improvement' of the hip osteoarthritis joint.
The female dog may produce colostrum, a type of milk high in nutrients and antibodies, 1–7 days before giving birth.
Milk production lasts for around three months, and increases with litter size.
The dog can sometimes vomit and refuse food during child contractions.
In the later stages of the dog's pregnancy, nesting behaviour may occur.
Puppies are born with a protective fetal membrane that the mother usually removes shortly after birth.
Dogs can have the maternal instincts to start grooming their puppies, consume their puppies' feces, and protect their puppies, likely due to their hormonal state.
While male-parent dogs can show more disinterested behaviour toward their own puppies, most can play with the young pups as they would with other dogs or humans.
A female dog may abandon or attack her puppies or her male partner dog if she is stressed or in pain.
Researchers have tested dogs' ability to perceive information, retain it as knowledge, and apply it to solve problems.
Studies of two dogs suggest that dogs can learn by inference.
A study with Rico, a Border Collie, showed that he knew the labels of over 200 different items.
He inferred the names of novel things by exclusion learning and correctly retrieved those new items after four weeks of the initial exposure.
A study of another Border Collie, Chaser, documented that he had learned the names and could associate them by verbal command with over 1,000 words.
One study of canine cognitive abilities found that dogs' capabilities are similar to those of horses, chimpanzees, or cats.
One study of 18 household dogs found that the dogs could not distinguish food bowls at specific locations without distinguishing cues; the study stated that this indicates a lack of spatial memory.
A study stated that dogs have a visual sense for number.
The dogs showed a ratio-dependent activation both for numerical values from 1–3 to larger than four.
Dogs demonstrate a theory of mind by engaging in deception.
Another experimental study showed evidence that Australian dingos can outperform domestic dogs in non-social problem-solving, indicating that domestic dogs may have lost much of their original problem-solving abilities once they joined humans.
Another study showed that dogs stared at humans after failing to complete an impossible version of the same task they had been trained to solve.
Dog communication is the transfer of information between dogs, as well as between dogs and humans.
Communication behaviors of dogs include eye gaze, facial expression, vocalization, body posture (including movements of bodies and limbs), and gustatory communication (scents, pheromones, and taste).
Dogs mark their territories by urinating on them, which is more likely when entering a new environment.
Both sexes of dogs may also urinate to communicate anxiety or frustration, submissiveness, or when in exciting or relaxing situations.
Aroused dogs can be a result of the dogs' higher cortisol levels.
Dogs begin socializing with other dogs by the time they reach the ages of 3 to 8 weeks, and at about 5 to 12 weeks of age, they alter their focus from dogs to humans.
Belly exposure in dogs can be a defensive behavior that can lead to a bite or to seek comfort.
Humans communicate with dogs by using vocalization, hand signals, and body posture.
With their acute sense of hearing, dogs rely on the auditory aspect of communication for understanding and responding to various cues, including the distinctive barking patterns that convey different messages.
A study using functional magnetic resonance imaging (fMRI) has shown that dogs respond to both vocal and nonvocal voices using the brain's region towards the temporal pole, similar to that of humans' brains.
Most dogs also looked significantly longer at the face whose expression matched the valence of vocalization.
A study of caudate responses shows that dogs tend to respond more positively to social rewards than to food rewards.
The dog is the most widely abundant large carnivoran living in the human environment.
In 2020, the estimated global dog population was between 700 million and 1 billion.
In the same year, a study found the dog to be the most popular pet in the United States, as they were present in 34 out of every 100 homes.
About 20% of the dog population live in developed countries.
In the developing world, it is estimated that three-quarters of the world's dog population lives in the developing world as feral, village, or community dogs.
Most of these dogs live as scavengers and have never been owned by humans, with one study showing that village dogs' most common response when approached by strangers is to run away (52%) or respond aggressively (11%).
Feral and free-ranging dogs' potential to compete with other large carnivores is limited by their strong association with humans.
Although wolves are known to kill dogs, wolves tend to live in pairs in areas where they are highly persecuted, giving them a disadvantage when facing large dog groups.
Coyotes and big cats have also been known to attack dogs.
In particular, leopards are known to have a preference for dogs and have been recorded to kill and consume them, no matter their size.
Siberian tigers in the Amur river region have killed dogs in the middle of villages.
Striped hyenas are known to kill dogs in their range.
Dogs as introduced predators have affected the ecology of New Zealand, which lacked indigenous land-based mammals before human settlement.
Dogs have made 11 vertebrate species extinct and are identified as a 'potential threat' to at least 188 threatened species worldwide.
Dogs have also been linked to the extinction of 156 animal species.
Dogs have been documented to have killed a few birds of the endangered species, the kagu, in New Caledonia.
Dogs are typically described as omnivores.
Compared to wolves, dogs from agricultural societies have extra copies of amylase and other genes involved in starch digestion that contribute to an increased ability to thrive on a starch-rich diet.
Similar to humans, some dog breeds produce amylase in their saliva and are classified as having a high-starch diet.
Despite being an omnivore, dogs are only able to conjugate bile acid with taurine.
They must get vitamin D from their diet.
Of the twenty-one amino acids common to all life forms (including selenocysteine), dogs cannot synthesize ten: arginine, histidine, isoleucine, leucine, lysine, methionine, phenylalanine, threonine, tryptophan, and valine.
Like cats, dogs require arginine to maintain nitrogen balance.
These nutritional requirements place dogs halfway between carnivores and omnivores.
As a domesticated or semi-domesticated animal, the dog has notable exceptions of presence in: The Aboriginal Tasmanians, who were separated from Australia before the arrival of dingos on that continent; The Andamanese peoples, who were isolated when rising sea levels covered the land bridge to Myanmar; The Fuegians, who instead domesticated the Fuegian dog, an already extinct different canid species; Individual Pacific islands whose maritime settlers did not bring dogs or where the dogs died out after original settlement, notably the Mariana Islands, Palau and most of the Caroline Islands with exceptions such as Fais Island and Nukuoro, the Marshall Islands, the Gilbert Islands, New Caledonia, Vanuatu, Tonga, Marquesas, Mangaia in the Cook Islands, Rapa Iti in French Polynesia, Easter Island, the Chatham Islands, and Pitcairn Island.
Dogs were introduced to Antarctica as sled dogs.
Starting practice in December 1993, dogs were later outlawed by the Protocol on Environmental Protection to the Antarctic Treaty international agreement due to the possible risk of spreading infections.
The domesticated dog originated as a predator and scavenger.
They inherited complex behaviors, such as bite inhibition, from their wolf ancestors, which would have been pack hunters with complex body language.
These sophisticated forms of social cognition and communication may account for dogs' trainability, playfulness, and ability to fit into human households and social situations, and probably also their co-existence with early human hunter-gatherers.
Dogs perform many roles for people, such as hunting, herding, pulling loads, protection, assisting police and the military, companionship, and aiding disabled individuals.
These roles in human society have earned them the nickname "man's best friend" in the Western world.
In some cultures, however, dogs are also a source of meat.
The keeping of dogs as companions, particularly by elites, has a long history.
Pet-dog populations grew significantly after World War II as suburbanization increased.
In the 1980s, there have been changes in the pet dog's functions, such as the increased role of dogs in the emotional support of their human guardians.
Within the second half of the 20th century, more and more dog owners considered their animal to be a part of the family.
This major social status shift allowed the dog to conform to social expectations of personality and behavior.
The second has been the broadening of the concepts of family and the home to include dogs-as-dogs within everyday routines and practices.
Products such as dog-training books, classes, and television programs target dog owners.
Some dog-trainers have promoted a dominance model of dog-human relationships.
Human family members have increased participation in activities in which the dog is an integral partner, such as dog dancing and dog yoga.
According to statistics published by the American Pet Products Manufacturers Association in the National Pet Owner Survey in 2009–2010, an estimated 77.5 million people in the United States have pet dogs.
The source shows that nearly 40% of American households own at least one dog, of which 67% own just one dog, 25% own two dogs, and nearly 9% own more than two dogs.
The data also shows an equal number of male and female pet dogs; less than one-fifth of the owned dogs come from shelters.
In addition to dogs' role as companion animals, dogs have been bred for herding livestock (such as collies and sheepdogs); for hunting; for rodent control (such as terriers); as search and rescue dogs; as detection dogs (such as those trained to detect illicit drugs or chemical weapons); as homeguard dogs; as police dogs (sometimes nicknamed "K-9"); as welfare-purpose dogs; as dogs who assist fishermen retrieve their nets; and as dogs that pull loads (such as sled dogs).
In 1957, the dog Laika became one of the first animals to be launched into Earth orbit aboard the Soviets's Sputnik 2; Laika died during the flight from overheating.
Various kinds of service dogs and assistance dogs, including guide dogs, hearing dogs, mobility assistance dogs, and psychiatric service dogs, assist individuals with disabilities.
A study of 29 dogs found that 9 dogs owned by people with epilepsy were reported to exhibit attention-getting behavior to their handler 30 seconds to 45 minutes prior to an impending seizure; there was no significant correlation between the patients' demographics, health, or attitude towards their pets.
Dogs compete in breed-conformation shows and dog sports (including racing, sledding, and agility competitions).
In dog shows, also referred to as "breed shows", a judge familiar with the specific dog breed evaluates individual purebred dogs for conformity with their established breed type as described in a breed standard.
Weight pulling, a dog sport involving pulling weight, has been criticized for promoting doping and for its risk of injury.
Humans have consumed dog meat going back at least 14,000 years.
For centuries, the practice was prevalent in Southeast Asia, East Asia, Africa, and Oceania before cultural changes triggered by the spread of religions resulted in dog meat consumption declining and becoming more taboo.
Switzerland, Polynesia, and pre-Columbian Mexico historically consumed dog meat.
Some Native American dogs, like the Peruvian Hairless Dog and Xoloitzcuintle, were raised to be sacrificed and eaten.
Han Chinese traditionally ate dogs.
Consumption of dog meat declined but did not end during the Sui dynasty (581–618) and Tang dynasty (618–907) due in part to the spread of Buddhism and the upper class rejecting the practice.
Dog consumption was rare in India, Iran, and Europe.
Eating dog meat is a social taboo in most parts of the world, though some still consume it in modern times.
It is still consumed in some East Asian countries, including China, Vietnam, Korea, Indonesia, and the Philippines.
An estimated 30 million dogs are killed and consumed in Asia every year.
China is the world's largest consumer of dogs, with an estimated 10 to 20 million dogs killed every year for human consumption.
In Vietnam, about 5 million dogs are slaughtered annually.
In 2024, China, Singapore, and Thailand placed a ban on the consumption of dogs within their borders.
In some parts of Poland and Central Asia, dog fat is reportedly believed to be beneficial for the lungs.
There is a long history of dog meat consumption in South Korea, but the practice has fallen out of favor.
On 9 January 2024, the South Korean parliament passed a law banning the distribution and sale of dog meat. It will take effect in 2027, with plans to assist dog farmers in transitioning to other products.
The primary type of dog raised for meat in South Korea has been the Nureongi.
In North Korea where meat is scarce, eating dog is a common and accepted practice, officially promoted by the government.
In 2018, the World Health Organization (WHO) reported that 59,000 people died globally from rabies, with 59.6% of the deaths in Asia and 36.4% in Africa. Rabies is a disease for which dogs are the most significant vector.
Dog bites affect tens of millions of people globally each year.
The primary victims of dog bite incidents are children.
They are more likely to sustain more serious injuries from bites, which can lead to death.
Sharp claws can lacerate flesh and cause serious infections.
In the United States, cats and dogs are a factor in more than 86,000 falls each year.
It has been estimated that around 2% of dog-related injuries treated in U.K. hospitals are domestic accidents.
The same study concluded that dog-associated road accidents involving injuries more commonly involve two-wheeled vehicles.
Some countries and cities have also banned or restricted certain dog breeds, usually for safety concerns.
Toxocara canis (dog roundworm) eggs in dog feces can cause toxocariasis.
Untreated toxocariasis can cause retinal damage and decreased vision.
Dog feces can also contain hookworms that cause cutaneous larva migrans in humans.
The scientific evidence is mixed as to whether a dog's companionship can enhance human physical and psychological well-being.
Studies suggest that there are benefits to physical health and psychological well-being, but they have been criticized for being "poorly controlled".
Earlier studies have shown that pet-dog or -cat guardians make fewer hospital visits and are less likely to be on medication for heart problems and sleeping difficulties than non-guardians.
People with pet dogs took considerably more physical exercise than those with cats or those without pets; these effects are relatively long-term.
Pet guardianship has also been associated with increased survival in cases of coronary artery disease.
Human guardians are significantly less likely to die within one year of an acute myocardial infarction than those who do not own dogs.
Studies have found a small to moderate correlation between dog-ownership and increased adult physical-activity levels.
Health benefits of dogs can result from contact with dogs in general, not solely from having dogs as pets.
For example, when in a pet dog's presence, people show reductions in cardiovascular, behavioral, and psychological indicators of anxiety and are exposed to immune-stimulating microorganisms, which can protect against allergies and autoimmune diseases (according to the hygiene hypothesis).
Other benefits include dogs as social support.
One study indicated that wheelchair-users experience more positive social interactions with strangers when accompanied by a dog than when they are not.
In a 2015 study, it was found that having a pet made people more inclined to foster positive relationships with their neighbors.
In one study, new guardians reported a significant reduction in minor health problems during the first month following pet acquisition, which was sustained through the 10-month study.
Using dogs and other animals as a part of therapy dates back to the late-18th century, when animals were introduced into mental institutions to help socialize patients with mental disorders.
Animal-assisted intervention research has shown that animal-assisted therapy with a dog can increase smiling and laughing among people with Alzheimer's disease.
One study demonstrated that children with ADHD and conduct disorders who participated in an education program with dogs and other animals showed increased attendance, knowledge, and skill-objectives and decreased antisocial and violent behavior compared with those not in an animal-assisted program.
Artworks have depicted dogs as symbols of guidance, protection, loyalty, fidelity, faithfulness, alertness, and love.
In ancient Mesopotamia, from the Old Babylonian period until the Neo-Babylonian period, dogs were the symbol of Ninisina, the goddess of healing and medicine, and her worshippers frequently dedicated small models of seated dogs to her.
In the Neo-Assyrian and Neo-Babylonian periods, dogs served as emblems of magical protection.
In China, Korea, and Japan, dogs are viewed as kind protectors.
In mythology, dogs often appear as pets or as watchdogs.
Stories of dogs guarding the gates of the underworld recur throughout Indo-European mythologies and may originate from Proto-Indo-European traditions.
In Greek mythology, Cerberus is a three-headed, dragon-tailed watchdog who guards the gates of Hades.
Dogs also feature in association with the Greek goddess Hecate.
In Norse mythology, a dog called Garmr guards Hel, a realm of the dead.
In Persian mythology, two four-eyed dogs guard the Chinvat Bridge.
In Welsh mythology, Cŵn Annwn guards Annwn.
In Hindu mythology, Yama, the god of death, owns two watchdogs named Shyama and Sharvara, which each have four eyes—they are said to watch over the gates of Naraka.
A black dog is considered to be the vahana (vehicle) of Bhairava (an incarnation of Shiva).
In Christianity, dogs represent faithfulness.
Within the Roman Catholic denomination specifically, the iconography of Saint Dominic includes a dog after the saint's mother dreamt of a dog springing from her womb and became pregnant shortly after that.
As such, the Dominican Order (Ecclesiastical Latin: Domini canis) means "dog of the Lord" or "hound of the Lord".
In Christian folklore, a church grim often takes the form of a black dog to guard Christian churches and their churchyards from sacrilege.
Jewish law does not prohibit keeping dogs and other pets but requires Jews to feed dogs (and other animals that they own) before themselves and to make arrangements for feeding them before obtaining them.
The view on dogs in Islam is mixed, with some schools of thought viewing them as unclean, although Khaled Abou El Fadl states that this view is based on "pre-Islamic Arab mythology" and "a tradition [...] falsely attributed to the Prophet".
The Sunni Maliki school jurists disagree with the idea that dogs are unclean.
Dog – the species (or subspecies) as a whole, also any male member of the same.
Bitch – any female member of the species (or subspecies).
Puppy or pup – a young member of the species (or subspecies) under 12 months old.
Sire – the male parent of a litter.
Dam – the female parent of a litter.
Litter – all of the puppies resulting from a single whelping.
Whelping – the act of a bitch giving birth.
Whelps – puppies still dependent upon their dam.

Based on:
  https://en.wikipedia.org/wiki/Dog

### Similarity with User's Preference

Now you can also rank the search results based on your own preference.

In [24]:
ans = input("Are you a dog or cat person? Please respond in 'cat' or 'dog'.\n")

Are you a dog or cat person? Please respond in 'cat' or 'dog'.
cat


Use Gemini API to get embeddings for your answer and compare them with search results

In [25]:
hypothetical_ans = create_embeddings([ans])

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity scores to rank the search results

In [26]:
sim_value = dot_product(search_res, hypothetical_ans)
sim_value

array([[0.53450373],
       [0.68786888],
       [0.67093936]])

using `np.argmax` best candidate is selected.

**Users's Input:** `dog/ cat`

**Answer:**

In [27]:
display(Markdown(summaries[np.argmax(sim_value)]))

*   The cat (Felis catus), also referred to as the domestic cat or house cat, is a small domesticated carnivorous mammal.
*   It is the only domesticated species of the family Felidae.
*   Domestication of the cat occurred in the Near East around 7500 BC.
*   It is commonly kept as a pet and working cat, but also ranges freely as a feral cat avoiding human contact.
*   It is valued by humans for companionship and its ability to kill vermin.
*   Its retractable claws are adapted to killing small prey species such as mice and rats.
*   It has a strong, flexible body, quick reflexes, and sharp teeth.
*   Its night vision and sense of smell are well developed.
*   It is a social species, but a solitary hunter and a crepuscular predator.
*   Cat intelligence is evident in their ability to adapt, learn through observation, and solve problems.
*   Research has shown they possess strong memories, exhibit neuroplasticity, and display cognitive skills comparable to those of a young child.
*   Cat communication includes meowing, purring, trilling, hissing, growling, grunting, and body language.
*   It can hear sounds too faint or too high in frequency for human ears, such as those made by small mammals.
*   It secretes and perceives pheromones.
*   Female domestic cats can have kittens from spring to late autumn in temperate zones and throughout the year in equatorial regions.
*   Litter sizes often range from two to five kittens.
*   Domestic cats are bred and shown at cat fancy events as registered pedigreed cats.
*   Population control includes spaying and neutering.
*   Pet abandonment has exploded the global feral cat population, which has driven the extinction of bird, mammal, and reptile species.
*   Domestic cats are found across the globe.
*   Out of the estimated 600 million cats worldwide, 400 million reside in Asia, including 58 million pet cats in China.
*   The United States leads in cat ownership with 73.8 million cats.
*   In the United Kingdom, approximately 10.9 million domestic cats are kept as pets.
*   The origin of the English word cat, Old English catt, is thought to be the Late Latin word cattus, first used at the beginning of the 6th century.
*   The Late Latin word may be derived from an unidentified African language (Nubian kaddîska, Nobiin kadīs are possible sources/cognates).
*   The forms might also have derived from an ancient Germanic word absorbed into Latin and then into Greek, Syriac, and Arabic.
*   The word may be derived from Germanic and Northern European languages, and ultimately be borrowed from Uralic (*käďwä, female (of a furred animal)).
*   The English puss, extended as pussy and pussycat, is attested from the 16th century and may have been introduced from Dutch poes or from Low German puuskatte.
*   Similar forms exist in Lithuanian puižė and Irish puisín or puiscín.
*   The etymology of "puss" is unknown, but it may be an onomatopoeia from using a sound to attract a cat.
*   A male cat is called a tom or tomcat (or a gib, if neutered).
*   A female is called a queen (or sometimes a molly, if spayed). Some sources write that queen refers solely to unspayed cats that are in an estrous cycle.
*   A juvenile cat is referred to as a kitten, a term interchangeable with the now-obsolete word catling in Early Modern English.
*   A group of cats can be referred to as a clowder, a glaring, or a colony.
*   The scientific name Felis catus was proposed by Carl Linnaeus in 1758 for a domestic cat.
*   Felis catus domesticus was proposed by Johann Christian Polycarp Erxleben in 1777.
*   Felis daemon proposed by Konstantin Satunin in 1904 was a black cat from the Transcaucasus, later identified as a domestic cat.
*   In 2003, the International Commission on Zoological Nomenclature ruled that the domestic cat is a distinct species, namely Felis catus.
*   In 2007, the modern domesticated subspecies F. silvestris catus sampled worldwide was considered to have probably descended from the African wildcat (F. lybica).
*   In 2017, the IUCN Cat Classification Taskforce followed the recommendation of the ICZN in regarding the domestic cat as a distinct species, Felis catus.
*   The domestic cat is a member of the Felidae, a family that has a common ancestor from about 10 to 15 million years ago.
*   The evolutionary radiation of the Felidae began in Asia during the Miocene around 8.38 to 14.45 million years ago.
*   Analysis of mitochondrial DNA of all Felidae species indicates a radiation at 6.46 to 16.76 million years ago.
*   The genus Felis genetically diverged from other Felidae around 6 to 7 million years ago.
*   Results of phylogenetic research shows that the wild members of this genus evolved through sympatric or parapatric speciation, whereas the domestic cat evolved through artificial selection.
*   The domestic cat and its closest wild ancestor are diploid and both possess 38 chromosomes and roughly 20,000 genes.
*   The earliest known indication for the taming of an African wildcat was excavated close by a human Neolithic grave in Shillourokambos, southern Cyprus, dating to about 7500–7200 BC.
*   African wildcats were attracted to early human settlements in the Fertile Crescent by rodents, in particular the house mouse (Mus musculus), and were tamed by Neolithic farmers.
*   The earliest known evidence for the occurrence of the domestic cat in Greece dates to around 1200 BC.
*   Greek, Phoenician, Carthaginian and Etruscan traders introduced domestic cats to southern Europe.
*   By the 5th century BC, they were familiar animals around settlements in Magna Graecia and Etruria.
*   During the Roman Empire, they were introduced to Corsica and Sardinia before the beginning of the 1st century AD.
*   By the end of the Western Roman Empire in the 5th century, the Egyptian domestic cat lineage had arrived in a Baltic Sea port in northern Germany.
*   The leopard cat (Prionailurus bengalensis) was tamed independently in China around 5500 BC, but this line leaves no trace in modern domestic cat populations.
*   During domestication, cats have undergone only minor changes in anatomy and behavior, and they are still capable of surviving in the wild.
*   Wildcat traits pre-adapted them for domestication: small size, social nature, obvious body language, love of play, and high intelligence.
*   Their rigorous grooming habits and instinct to bury their bodily waste make them generally much less messy than other domesticated animals.
*   House cats may mate with feral cats.
*   Hybridization between domestic and other Felinae species is also possible (e.g., Kellas cat).
*   Development of cat breeds started in the mid-19th century.
*   An analysis of the domestic cat genome revealed that the ancestral wildcat genome was significantly altered in the process of domestication, as specific mutations were selected to develop cat breeds.
*   Most breeds are founded on random-bred domestic cats.
*   Genetic diversity of these breeds varies between regions, and is lowest in purebred populations, which show more than 20 deleterious genetic disorders.
*   The domestic cat has a smaller skull and shorter bones than the European wildcat.
*   It averages about 46 cm (18 in) in head-to-body length and 23–25 cm (9.1–9.8 in) in height, with about 30 cm (12 in) long tails.
*   Males are larger than females.
*   Adult domestic cats typically weigh 4–5 kg (8.8–11.0 lb).
*   Cats have seven cervical vertebrae, 13 thoracic vertebrae, seven lumbar vertebrae, three sacral vertebrae, and a variable number of caudal vertebrae in the tail.
*   The extra lumbar and thoracic vertebrae account for the cat's spinal mobility and flexibility.
*   Attached to the spine are 13 ribs, the shoulder, and the pelvis.
*   Cat forelimbs are attached to the shoulder by free-floating clavicle bones.
*   The cat skull is unusual among mammals in having very large eye sockets and a powerful specialized jaw.
*   Two long canine teeth for killing and tearing prey, can stab between two of the prey's vertebrae and sever its spinal cord.
*   Domestic cats have narrowly spaced canine teeth relative to the size of their jaw, an adaptation to their preferred prey of small rodents.
*   The premolar and first molar together compose the carnassial pair on each side of the mouth, which efficiently shears meat.
*   Cats' small molars cannot chew food effectively, and cats are largely incapable of mastication.
*   Cats tend to have better dental health than most humans.
*   Cats have protractible and retractable claws.
*   Claws are sheathed in their normal, relaxed position, keeping them sharp and allowing silent stalking.
*   The claws on the forefeet are typically sharper than those on the hindfeet.
*   Cats can voluntarily extend their claws, such as in hunting, fighting, climbing, kneading, or for extra traction on soft surfaces.
*   Cats shed the outside layer of their claw sheaths when scratching rough surfaces.
*   Most cats have five claws on their front paws and four on their rear paws.
*   The dewclaw is proximal to the other claws.
*   A protrusion on the front paws inside the wrists appears to be a sixth "finger," thought to be an antiskidding device used while jumping.
*   Some cat breeds are prone to having extra digits ("polydactyly").
*   The cat is digitigrade, walking on its toes.
*   It uses a "pacing" gait that alternates both legs together on each side.
*   It registers directly by placing each hind paw close to the track of the corresponding fore paw, minimizing noise and visible tracks.
*   As it speeds up from walking to trotting, its gait changes to a "diagonal" gait.
*   Cats are generally fond of perching in high places.
*   A cat falling from up to 3 m (9.8 ft) can right itself and land on its paws.
*   During a lofty fall, a cat reflexively twists and rights itself to land on its feet using its acute sense of balance and flexibility; this is known as the cat righting reflex.
*   A cat always rights itself in the same way, and it has enough time in falls of at least 90 cm (3.0 ft).
*   The cat family (Felidae) can pass down many colors and patterns to their offspring.
*   The domestic cat genes MC1R and ASIP allow color variety in their coats.
*   Cats have excellent night vision and can see at one sixth the light level required for human vision.
*   Cat eyes have a tapetum lucidum, which reflects light back into the eye, increasing sensitivity to dim light.
*   Large pupils are an adaptation to dim light.
*   The domestic cat has slit pupils, which allow it to focus bright light without chromatic aberration.
*   At low light, a cat's pupils expand to cover most of the exposed surface of its eyes.
*   The domestic cat has rather poor color vision and only two types of cone cells (blue and yellowish green).
*   Its ability to distinguish between red and green is limited.
*   Cats have a nictitating membrane, allowing them to blink without hindering their vision.
*   The domestic cat's hearing is most acute in the range of 500 Hz to 32 kHz.
*   It can detect an extremely broad range of frequencies ranging from 55 Hz to 79 kHz.
*   It can hear a range of 10.5 octaves, compared to about 9 octaves for humans and dogs.
*   Its hearing sensitivity is enhanced by its large movable outer ears (pinnae).
*   It can detect ultrasound, including ultrasonic calls from rodent prey.
*   Cats have socio-spatial cognitive abilities to create mental maps of familiar people's locations based on hearing their voices.
*   Cats have an acute sense of smell, due to their well-developed olfactory bulb and a large surface of olfactory mucosa (about 5.8 cm2).
*   Cats and many other animals have a Jacobson's organ in their mouths that is used in flehmening to sense certain aromas in a way humans cannot.
*   Cats are sensitive to pheromones such as 3-mercapto-3-methylbutan-1-ol, which they use to communicate through urine spraying and marking with scent glands.
*   About 70–80% of cats are affected by nepetalactone (in catnip, silver vine, valerian).
*   Cats have about 470 taste buds, compared to more than 9,000 on the human tongue.
*   Domestic and wild cats share a taste receptor gene mutation that keeps their sweet taste buds from binding to sugary molecules, leaving them with no ability to taste sweetness.
*   They do have taste bud receptors specialized for acids, amino acids (constituents of protein), and bitter tastes.
*   Cats' taste buds possess the receptors needed to detect umami, detecting inosine monophosphate and histidine (particularly enriched in tuna).
*   Cats distinctly prefer food temperature around 38 °C (100 °F), similar to a fresh kill.
*   Some cats reject cold food, which would signal to the cat that the prey is long dead and therefore possibly toxic or decomposing.
*   To aid with navigation and sensation, cats have dozens of movable whiskers (vibrissae) over their body, especially their faces.
*   Whiskers provide information on the width of gaps and on the location of objects in the dark, by touching objects directly and by sensing air currents.
*   They trigger protective blink reflexes to protect the eyes from damage.
*   Outdoor cats are active both day and night, although they tend to be slightly more active at night.
*   Domestic cats spend the majority of their time in the vicinity of their homes, but they can range a radius of many hundreds of meters.
*   They establish territories that vary considerably in size, in one study ranging 7–28 ha (17–69 acres).
*   Being low-light predators, they are generally crepuscular (more active near dawn and dusk).
*   House cats' behavior is also influenced by human activity.
*   Cats conserve energy by sleeping more than most animals, especially as they grow older.
*   The daily duration of sleep varies, usually between 12 and 16 hours, with 13 to 14 being the average.
*   Some cats can sleep as much as 20 hours.
*   The term "cat nap" for a short rest refers to the cat's tendency to fall asleep (lightly) for a brief period.
*   Short periods of rapid eye movement sleep are often accompanied by muscle twitches, suggesting dreaming.
*   Behavioral and personality traits depend on a complex interplay between genetic and environmental factors.
*   The social behavior of the domestic cat ranges from widely dispersed individuals to feral cat colonies that gather around a food source, based on groups of co-operating females.
*   Within such groups, one cat is usually dominant over the others.
*   Each cat in a colony holds a distinct territory, with sexually active males having the largest territories (about 10 times larger than females' and may overlap with several females' territories).
*   These territories are marked by urine spraying, rubbing objects at head height with secretions from facial glands, and by defecation.
*   Between these territories are neutral areas where cats watch and greet one another without territorial conflicts.
*   Outside neutral areas, territory holders usually chase away stranger cats, initially by staring, hissing, and growling, and then by short and violent, noisy attacks.
*   Although cats do not have a social survival strategy or herd behavior, they always hunt alone.
*   Life in proximity to humans and other domestic animals has led to a symbiotic social adaptation in cats.
*   Cats may express great affection toward humans or other animals.
*   Ethologically, a cat's human keeper functions as a mother surrogate.
*   Adult cats live in a type of extended kittenhood, a form of behavioral neoteny.
*   Their high-pitched sounds may mimic the cries of a hungry human infant.
*   Some pet cats are poorly socialized and older cats show aggressiveness toward newly arrived kittens.
*   Redirected aggression is a common form of aggression in multiple cat households, where agitation towards one stimulus is directed at another cat, pet, or human.
*   Domestic cats' scent rubbing behavior toward humans or other cats is thought to be a feline means of social bonding.
*   Domestic cats use many vocalizations for communication, including purring, trilling, hissing, growling/snarling, grunting, and several different forms of meowing.
*   Their body language, including position of ears and tail, relaxation of the whole body, and kneading of the paws, are all indicators of mood.
*   The tail and ears are particularly important social signal mechanisms; a raised tail indicates a friendly greeting, and flattened ears indicate hostility.
*   Tail-raising also indicates the cat's position in the group's social hierarchy, with dominant individuals raising their tails less often than subordinate ones.
*   Feral cats are generally silent.
*   Nose-to-nose touching is also a common greeting and may be followed by social grooming.
*   Purring may have developed as an evolutionary advantage as a signaling mechanism of reassurance between mother cats and nursing kittens, and is thought to be a care-soliciting signal.
*   Post-nursing cats also often purr as a sign of contentment.
*   Purring has been recorded in a wide variety of circumstances, most of which involve physical contact between the cat and another, presumably trusted individual.
*   Some cats purr continuously when chronically ill or in apparent pain.
*   Purring is proposed to be generated via a series of sudden build-ups and releases of pressure as the glottis is opened and closed, causing the vocal folds to separate forcefully.
*   The laryngeal muscles in control of the glottis are thought to be driven by a neural oscillator which generates a cycle of contraction and release every 30–40 milliseconds (frequency of 33 to 25 Hz).
*   Domestic cats observed in rescue facilities have 276 morphologically distinct facial expressions based on 26 facial movements.
*   Facial expressions have helped researchers detect pain in cats, using the feline grimace scale's five criteria: ear position, orbital tightening, muzzle tension, whisker change, and head position.
*   Cats are known for spending considerable amounts of time licking their coats to keep them clean.
*   The cat's tongue has backward-facing spines about 0.5 millimeter long, called lingual papillae, which contain keratin making them rigid and act like a hairbrush.
*   Some cats, particularly long-haired cats, occasionally regurgitate sausage-shaped 2–3 cm (0.8–1.2 in) long hairballs of fur.
*   Hairballs can be prevented with remedies that ease elimination of the hair through the gut, and regular grooming.
*   Cat intelligence refers to a cat's ability to solve problems, adapt to its environment, learn new behaviors, and communicate its needs.
*   A cat's brain contains around 250 million neurons in the cerebral cortex.
*   Cats display neuroplasticity.
*   They have well-developed memory retaining information for a decade or longer.
*   These memories are often intertwined with emotions, allowing cats to recall both positive and negative experiences.
*   While they excel in observational learning and problem-solving, studies conclude that they struggle with understanding cause-and-effect relationships in the same way that humans do.
*   Cat intelligence study is mostly from consideration of the domesticated cat.
*   Living in urban environments has exposed them to challenges that require adaptive behaviors, contributing to cognitive development.
*   Selective breeding and genetic changes have further influenced their intelligence.
*   Kittens learn essential survival skills by observing their mothers, while adult cats refine their abilities through trial and error.
*   Domestic cats, especially young kittens, are known for their love of play.
*   This behavior mimics hunting and is important in helping kittens learn to stalk, capture, and kill prey.
*   Cats also engage in play fighting, both with each other and with humans.
*   Cats tend to play with toys more when they are hungry.
*   Cats prefer to play with objects that resemble prey, such as small furry toys that move rapidly.
*   They become habituated to a toy they have played with before.
*   String is often used as a toy, but if eaten, it can cause a medical emergency.
*   The shape and structure of cats' cheeks is insufficient to allow them to take in liquids using suction.
*   Lapping at a rate of four times a second, the cat touches the smooth tip of its tongue to the surface of the water, and quickly retracts it like a corkscrew, drawing water upward into their mouths.
*   Feral cats and free-fed house cats consume several small meals in a day.
*   They select food based on its temperature, smell, and texture; they dislike chilled foods and respond most strongly to moist foods rich in amino acids.
*   Cats reject novel flavors (neophobia) and learn quickly to avoid foods that have tasted unpleasant.
*   Most adult cats are lactose intolerant.
*   Some develop pica, eating or chewing on things such as wool, plastic, cables, paper, string, aluminum foil, or even coal.
*   Cats hunt small prey, primarily birds and rodents, and are often used as a form of pest control.
*   Other common small creatures, such as lizards and snakes, may also become prey.
*   Cats use two hunting strategies: stalking prey actively, or waiting in ambush.
*   Domestic cats are a major predator of wildlife in the United States, killing an estimated 1.3 to 4.0 billion birds and 6.3 to 22.3 billion mammals annually.
*   In the recovery of ringed robins and dunnocks in Britain, 31% of deaths were a result of cat predation.
*   The presentation of prey to human guardians is a poorly understood element of cat hunting behavior.
*   Domestic males are more likely to fight than females.
*   The most common reason for feral cat fighting is competition between two males to mate with a female, and most fights are won by the heavier male.
*   Another common reason for fighting in domestic cats is the difficulty of establishing territories within a small home.
*   Female cats also fight over territory or to defend their kittens.
*   Neutering decreases or eliminates fighting behavior in many cases, suggesting a link to sex hormones.
*   When cats become aggressive, they try to appear larger and more threatening by raising their fur, arching their backs, turning sideways, hissing, or spitting.
*   Often, the ears are pointed down and back to avoid damage and listen for changes.
*   Cats may also vocalize loudly and bare their teeth.
*   Fights usually consist of grappling, slapping the face and body with the forepaws, and bites.
*   Cats throw themselves to the ground in a defensive posture to rake their opponent's belly with their hind legs.
*   Serious damage from fights is rare, usually limited to infections from scratches and bites.
*   Bites are probably the main route of transmission of the feline immunodeficiency virus.
*   Sexually active males are usually involved in many fights and have battered faces.
*   Cats are willing to threaten animals larger than them to defend their territory, such as dogs and foxes.
*   The cat secretes and perceives pheromones.
*   Female cats, called queens, are polyestrous with several estrus cycles during a year, lasting usually 21 days.
*   They are usually ready to mate between early February and August in northern temperate zones and throughout the year in equatorial regions.
*   Several males, called tomcats, are attracted to a female in heat and fight over her.
*   A male cat's penis has a band of about 120–150 backward-pointing penile spines, which are about 1 mm (0.04 in) long, inducing ovulation upon withdrawal.
*   After mating, the female cleans her vulva thoroughly and attacks males attempting to mate shortly after.
*   Cats are superfecund; a female may mate with more than one male, resulting in different kittens in a litter having different fathers.
*   The morula forms 124 hours after conception.
*   At 148 hours, early blastocysts form.
*   At 10–12 days, implantation occurs.
*   The gestation of queens lasts between 64 and 67 days, with an average of 65 days.
*   A study of 2,300 free-ranging queens found they had one to six kittens per litter, with an average of three kittens.
*   They produced a mean of 1.4 litters per year, but a maximum of three litters in a year.
*   Of 169 kittens, 127 died before they were six months old due to trauma (dog attacks and road accidents).
*   The first litter is usually smaller than subsequent litters.
*   Kittens are weaned between six and seven weeks of age.
*   Queens normally reach sexual maturity at 5–10 months, and males at 5–7 months, varying by breed.
*   Kittens reach puberty at the age of 9–10 months.
*   Cats are ready to go to new homes at about 12 weeks of age, when they are ready to leave their mother.
*   They can be surgically sterilized (spayed or castrated) as early as seven weeks to limit unwanted reproduction.
*   This surgery also prevents undesirable sex-related behavior, such as aggression, territory marking (spraying urine) in males, and yowling (calling) in females.
*   Traditionally, this surgery was performed at around six to nine months of age, but is increasingly performed before puberty, at about three to six months.
*   In the United States, about 80% of household cats are neutered.
*   The average lifespan of pet cats has risen in recent decades: about 7 years in the early 1980s, 9.4 years in 1995, and an average of about 13 years as of 2014 and 2023.
*   Neutering increases life expectancy; castrated male cats live twice as long as intact males, while spayed female cats live 62% longer than intact females.
*   Neutering confers some health benefits, such as a greater life expectancy and a decreased incidence of reproductive neoplasia.
*   However, neutering decreases metabolism and increases food intake, both of which can cause obesity.
*   About 250 heritable genetic disorders have been identified in cats; many are similar to human inborn errors of metabolism.
*   The high level of similarity among the metabolism of mammals allows many feline diseases to be diagnosed using genetic tests developed for humans.
*   Cats can be used as animal models in the study of human diseases.
*   Diseases affecting domestic cats include acute infections, parasitic infestations, injuries, and chronic diseases such as kidney disease, thyroid disease, and arthritis.
*   Vaccinations are available for many infectious diseases, as are treatments to eliminate parasites.
*   The domestic cat is a cosmopolitan species and occurs across much of the world.
*   It is adaptable and now present on all continents except Antarctica, and on 118 of the 131 main groups of islands.
*   Due to its ability to thrive in almost any terrestrial habitat, it is among the world's most invasive species.
*   It lives on small islands with no human inhabitants.
*   Feral cats can live in forests, grasslands, tundra, coastal areas, agricultural land, scrublands, urban areas, and wetlands.
*   It can readily interbreed with the wildcat, posing a danger to the genetic distinctiveness of some wildcat populations (e.g., Scotland, Hungary, Iberian Peninsula).
*   Its introduction to places where no native felines are present also contributes to the decline of native species.
*   Feral cats are domestic cats that were born in or have reverted to a wild state.
*   They are unfamiliar with and wary of humans and roam freely in urban and rural areas.
*   Estimates of the United States feral population range from 25 to 60 million.
*   Feral cats may live alone, but most are in large colonies, occupying a specific territory and usually associated with a source of food.
*   Famous feral cat colonies are in Rome around the Colosseum and Forum Romanum.
*   On islands, birds can contribute as much as 60% of a cat's diet.
*   Domestic cats are a contributing factor to the decline of several species, leading to extinction in some cases (e.g., South Island piopio, Chatham rail, New Zealand merganser, Lyall's wren).
*   One feral cat in New Zealand killed 102 New Zealand lesser short-tailed bats in seven days.
*   In Australia, one study found feral cats to kill 466 million reptiles per year.
*   More than 258 reptile species were identified as being predated by cats.
*   Cats have contributed to the extinction of the Navassa curly-tailed lizard and Chioninia coctei.
*   Cats are common pets throughout the world, and their worldwide population as of 2007 exceeded 500 million.
*   As of 2024, the domestic cat was the second most popular pet in the United States, with 73.8 million cats owned and around 42.2 million households owning at least one cat.
*   In the United Kingdom, 26% of adults have a cat, with an estimated population of 10.9 million pet cats as of 2020.
*   As of 2021, there were an estimated 220 million owned and 480 million stray cats in the world.
*   Cats have been used for millennia to control rodents, notably around grain stores and aboard ships.
*   Cats are also used in the international fur trade and leather industries for making coats, hats, blankets, stuffed toys, shoes, gloves, and musical instruments.
*   About 24 cats are needed to make a cat-fur coat.
*   This use has been outlawed in the United States since 2000 and in the European Union (as well as the United Kingdom) since 2007.
*   Cat pelts have been used for superstitious purposes as part of the practice of witchcraft, and are still made into blankets in Switzerland as traditional medicines.
*   General estimates for the global population of domestic cats range widely from anywhere between 200 million to 600 million.
*   Walter Chandoha made his career photographing cats after his 1949 images of Loco, a stray cat, were published.
*   He is reported to have photographed 90,000 cats during his career and maintained an archive of 225,000 images.
*   Pet humanization is a form of anthropomorphism in which cats are kept for companionship and treated more like human family members.
*   A cat show is a judged event in which the owners of cats compete to win titles in various cat-registering organizations by entering their cats to be judged after a breed standard.
*   It is often required that a cat must be healthy and vaccinated to participate in a cat show.
*   Both pedigreed and non-purebred companion ("moggy") cats are admissible.
*   Competing cats are compared to the applicable breed standard, and assessed for temperament.
*   Cats can be infected or infested with viruses, bacteria, fungus, protozoans, arthropods, or worms that can transmit diseases to humans.
*   The likelihood that a person will become diseased depends on the age and immune status of the person.
*   Humans who have cats living in their home or in close association are more likely to become infected.
*   Others might also acquire infections from cat feces and parasites exiting the cat's body.
*   Some of the infections of most concern include salmonella, cat-scratch disease, and toxoplasmosis.
*   In ancient Egypt, cats were revered, and the goddess Bastet often depicted in cat form.
*   The Greek historian Herodotus reported that killing a cat was forbidden, and when a household cat died, the entire family mourned and shaved their eyebrows.
*   Families took their dead cats to the sacred city of Bubastis, where they were embalmed and buried.
*   Ancient Greeks and Romans kept weasels as pets, which were seen as the ideal rodent-killers.
*   The earliest unmistakable evidence of the Greeks having domestic cats comes from two coins from Magna Graecia dating to the mid-fifth century BC.
*   The usual ancient Greek word for 'cat' was ailouros, meaning 'thing with the waving tail'.
*   Cats are rarely mentioned in ancient Greek literature.
*   Aristotle remarked in his History of Animals that "female cats are naturally lecherous".
*   The Greeks later syncretized their own goddess Artemis with the Egyptian goddess Bastet, adopting Bastet's associations with cats.
*   In Ovid's Metamorphoses, when the deities flee to Egypt and take animal forms, the goddess Diana turns into a cat.
*   Cats eventually displaced weasels as the pest control of choice.
*   During the Middle Ages, many of Artemis's associations with cats were grafted onto the Virgin Mary.
*   Cats are often shown in icons of Annunciation and of the Holy Family.
*   According to Italian folklore, on the same night that Mary gave birth to Jesus, a cat in Bethlehem gave birth to a kitten.
*   Domestic cats were spread throughout much of the rest of the world during the Age of Discovery, as ships' cats were carried on sailing ships to control shipboard rodents and as good-luck charms.
*   Several ancient religions believed cats are exalted souls, companions or guides for humans, that are all-knowing but mute.
*   In Japan, the maneki neko cat is a symbol of good fortune.
*   In Norse mythology, Freyja, the goddess of love, beauty, and fertility, is depicted as riding a chariot drawn by cats.
*   In Jewish legend, the first cat was living in the house of the first man Adam as a pet that got rid of mice.
*   Although no species are sacred in Islam, cats are revered by Muslims.
*   One of the companions of Muhammad was known as Abu Hurayrah ("father of the kitten"), in reference to his documented affection to cats.
*   Many cultures have negative superstitions about cats.
*   For example, that encountering a black cat ("crossing one's path") leads to bad luck, or that cats are witches' familiar spirits.
*   The killing of cats in medieval Ypres, Belgium, is commemorated in the present-day Kattenstoet (cat parade).
*   In mid-16th century France, cats were allegedly burnt alive as a form of entertainment, particularly during midsummer festivals.
*   According to a myth in many cultures, cats have multiple lives.
*   In many countries, they are believed to have nine lives, but in Italy, Germany, Greece, Brazil, and some Spanish-speaking regions, they are said to have seven lives, while in Arabic traditions, the number of lives is six.
*   An early mention of the myth is in John Heywood's The Proverbs of John Heywood (1546).
*   The myth is attributed to the natural suppleness and swiftness cats exhibit to escape life-threatening situations.
*   Falling cats often land on their feet, using an instinctive righting reflex.

Based on:
  https://en.wikipedia.org/wiki/Cat

You have now created a search re-ranking engine using embeddings based on your own preference!

## Next steps

I hope you found this example helpful! Check out more examples in the [Cookbook](https://github.com/google-gemini/cookbook) to learn more.